In [193]:
%pylab ipympl
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import math as mt
import collections
import pylab
import os
import glob
from scipy.interpolate import interp2d
from scipy.linalg import dft
from scipy.special import erf, erfc

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.6/dist-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['xlabel', 'ylabel', 'pylab', 'legend', 'e']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [194]:
print(os.getcwd())
origdir=os.getcwd()
Msun=1.98847*10**6

/user/dataA_rpod4/Huaqing/ppmstar/projects/H-core-M25/analysis_Prad-runs


In [89]:
#@input: r,rho,fv,airmu,cldmu,isflat
#@output: mass and cloud mass arrays
def get_mass(r,rho,fv,airmu,cldmu,isflat):
    sgn=1
    if r[0] > r[-1]:
        sgn = -1.
        r=np.flip(r)
        rho=np.flip(rho)
        fv=np.flip(fv)

    mu = fv*cldmu + (1.-fv)*airmu
    rhocld = (rho/mu)*cldmu 
    if isflat==1:
        dv = np.gradient(r)
    else:
        dv = np.gradient(4./3.*np.pi*r**3)
 
    m = np.cumsum(dv*rho)
    cldm = np.cumsum(dv*rhocld*fv)
    
    if sgn == -1.:
        r=np.flip(r)
        rho=np.flip(rho)
        fv=np.flip(fv)
        m=np.flip(m)
        cldm=np.flip(cldm)  

    return [m,cldm]

def line_plot(x,y,title,xlabel,ylabel,lab,sty):
    if title!=None:
        plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.plot(x,y,label=lab,markersize=6,markevery=30,linestyle=sty[0], marker=sty[1], color=sty[2])
    plt.legend(loc='upper left')        
    return 0 
                  
def smooth(y,npt):
    size=len(y)
    y_smooth=[0 for i in range(0,size)]
    for i in range(0,size):
        ct=0
        s=0
        for j in range(-npt,npt+1):
            if i+j>=0 and i+j<=(size-1):
                ct=ct+1
                s=s+y[i+j]
        y_smooth[i]=s/ct
    return np.array(y_smooth) 
    

def find_in_list(element,array):
    dev = [abs(array[i]-element) for i in range(0,len(array))]
    return dev.index(min(dev))

def style_gen():
    sty=[]
    lsty=['solid','dotted','dashed','dashdot',
          (0,(3,1,1,1)),(0,(5,1)),(0,(3,5,1,5)),(0,(3,1,1,1,1,1)),(0,(3,5,1,5))]
    color=['blue', 'forestgreen', 'deepskyblue',
                  'cornflowerblue', 'lightcoral', 'red',
                  'gold', 'darkgoldenrod', 'maroon','grey','black','lightsteelblue']
    mk=['D','d','s','*','v','<','>','^','+','.','p','x','h','H','_','X']
    sty.append([lsty[0],mk[0],color[0]])
    sty.append([lsty[1],mk[1],color[1]])
    sty.append([lsty[2],mk[2],color[2]])
    sty.append([lsty[3],mk[3],color[3]])
    sty.append([lsty[4],mk[4],color[4]])
    sty.append([lsty[5],mk[5],color[5]])
    sty.append([lsty[6],mk[6],color[6]])
    sty.append([lsty[7],mk[7],color[7]])
    sty.append([lsty[8],mk[8],color[8]])
    sty.append([lsty[0],mk[9],color[9]])
    sty.append([lsty[1],mk[10],color[10]])
    sty.append([lsty[2],mk[11],color[11]])
    sty.append([lsty[3],mk[13],color[11]])
    sty.append([lsty[4],mk[3],color[2]])
    sty.append([lsty[5],mk[4],color[3]])
    sty.append([lsty[6],mk[5],color[4]])
    sty.append([lsty[7],mk[6],color[5]])
    sty.append([lsty[8],mk[7],color[6]])
    
    return sty

In [90]:
#@input: a opened file object of rprof
#@output: dictionary of parameters and a list of dictionary {'R':array{1,2,...},'P1':array{1,2,...},...}
def getrprof(file):
# now read in some parameters from the rprof header
    params = {}
    while True:
        line = file.readline()
        if len(line) > 1:
            if line.split()[0] == 'DUMP':
                params['DUMP'] = float(line.split()[1])
                params['time'] = float(line.split()[11].split(',')[0])
                break
    while True:
        line = file.readline()
        if len(line) > 1:
            if line.split()[0] == 'dt':
                params['dt'] = float(line.split()[2].split(',')[0])
                break
    while True:
        line = file.readline()
        if len(line) > 1:
            if line.split()[0] == 'Nx':
                params['Nx'] = float(line.split()[2])
                break  
# now proceed to reading rprofs
    rprofs = []
    rprof = []
    ifbegin = False
    while True:
        line = file.readline()
        if len(line) > 1:
            if  (line.split()[0] == 'IR' or line.split()[0] == 'IRcyl'):
                ifbegin = True
                rprof = []
                rprof.append(line.split())
            elif (line.split()[0]=='DATE:' or line.split()[0]=='Line') :
                break
            elif ifbegin == True:
                row_of_data = [float(e) for e in line.split()]
                rprof.append(row_of_data)  
                if line.split()[0] == '1':
                    rprofs.append(rprof)
                    ifbegin = False  
# rprofs[i] is [[var1,var2,...],[1,2,...],[2,2,...]]
#Later, rprofs will be converted to a list of dictionaries. 

# now read in some parameters after rprofs
    while True:
        line = file.readline()
        if line == '':
            break
        split = line.split()
        for i in range(0,len(split)-1):
            if split[i] == 'airmu': params['airmu'] = float(split[i+1])
            if split[i] == 'cldmu': params['cldmu'] = float(split[i+1])
            if split[i] == 'radout0': params['radout0'] = float(split[i+1])
            if split[i] == 'radin0': params['radin0'] = float(split[i+1])
            if split[i] == 'radmax': params['radmax'] = float(split[i+1])
            if split[i] == 'fkair': params['fkair'] = float(split[i+1])
            if split[i] == 'fkcld': params['fkcld'] = float(split[i+1])
            if split[i] == 'radconst': params['radconst'] = float(split[i+1])
            if split[i] == 'Rgasconst': params['Rgasconst'] = float(split[i+1])
            if split[i] == 'totallum': params['totallum'] = float(split[i+1])
            if split[i] == 'gravconst': params['gravconst'] = float(split[i+1])
    params['rhocldtoair0'] = params['cldmu'] /params['airmu']
    params['dx'] = params['radmax'] *2./params['Nx']
    
    n = len(rprofs)
    res=[]
    for i in range(n):
        panel = rprofs[i]
        nrow,nvar = len(panel),len(panel[0])
        rprof = collections.defaultdict(list)
        var = panel[0]
        for r in range(1,nrow):
            row = panel[r]
            for iv in range(nvar):
                rprof[var[iv]].append(row[iv])
        for key in rprof:
            rprof[key] = np.array(rprof[key])
        res.append(rprof)

    return [params,res]
# End of definition of getrprof
#rprofs is a list of dictionaries in that, for instance, rprof[0] consist of 
# entries like 'IR':[1,2,...,128] .

path='/home/user/user/frontera_scratch3_mao1'
run='/M207-1000xht-1000xdiff-realEOS-1152'
params={}
rprofs=[]
file=open(path+run+'/prfs/HcoreM025Z0-4218.rprof','rt')
params,rprofs=getrprof(file)
print(params.keys())
print(rprofs[0].keys())
print(os.getcwd())
print(origdir)

dict_keys(['DUMP', 'time', 'dt', 'Nx', 'airmu', 'cldmu', 'fkair', 'fkcld', 'Rgasconst', 'gravconst', 'totallum', 'radin0', 'radout0', 'radmax', 'rhocldtoair0', 'dx'])
dict_keys(['IR', 'R', 'Rho0', 'Rho1', 'P0', 'P1', 'FV0', 'FV', 'T9'])
/user/dataA_rpod4/Huaqing/ppmstar/projects/H-core-M25/analysis_Prad-runs
/user/dataA_rpod4/Huaqing/ppmstar/projects/H-core-M25/analysis_Prad-runs


In [91]:
#@input: a file object 
#@output: a list of dictionary, each dictionary is {"var":array([100,99,...,1])}
def getmomsprof(file):         
    rprofs = []
    rprof = []
    ifbegin = False
    while True:
        line = file.readline()
        if ("" == line):
            break
        if len(line) > 1:
            if  (line.split()[0] == 'ir'):
                ifbegin = True
                rprof = []
                rprof.append(line.split())
            elif ifbegin == True:
                row_of_data = [float(e) for e in line.split()]
                rprof.append(row_of_data)  
                if line.split()[0] == '1':
                    rprofs.append(rprof)
                    ifbegin = False
# now rprofs is a list of rprof panels
# convert it a list of dictionary
    n = len(rprofs)
    res= []
    for i in range(n):
        panel=rprofs[i]
        nr,nvar = len(panel),len(panel[0])
        rprof = collections.defaultdict(list)
        var = panel[0]
        for ivar in range(nvar):
            data_list = np.array([])
            for ii in range(1,nr):
                data_list=np.append(data_list,panel[ii][ivar])
            key = var[ivar]+str('1') if (var[ivar] in rprof.keys()) else var[ivar]
            rprof[key] = data_list
        res.append(rprof)
    return res

print(origdir)
path='/home/user/user/frontera_scratch3_mao1'
run='/M207-1000xht-1000xdiff-realEOS-1152'
file=open(path+run+'/prfs/frombqavs/HcoreM025Z0-BQav2000.rprof','rt')
rprofs=getmomsprof(file)
print(rprofs[4].keys())
print(type(rprofs[4]['UdP1max']))
print(os.getcwd())
print(origdir)

/user/dataA_rpod4/Huaqing/ppmstar/projects/H-core-M25/analysis_Prad-runs
dict_keys(['ir', 'fvlg', 'fvlgmin', 'fvlgmax', 'fvlgdrms', 'UdP1', 'UdP1min', 'UdP1max', 'UdP1drms'])
<class 'numpy.ndarray'>
/user/dataA_rpod4/Huaqing/ppmstar/projects/H-core-M25/analysis_Prad-runs
/user/dataA_rpod4/Huaqing/ppmstar/projects/H-core-M25/analysis_Prad-runs


In [92]:
def avmomsprof(run,dmprange):
    runname=run.split('/')[-1].split('-')[0]
    print('run:',runname)
    os.chdir(run+'/prfs/frombqavs')
    prfs = glob.glob('*.rprof')
    ndumps = len(prfs)
    runid = prfs[0][:-10]
    start,end,inc = dmprange
    avmomsprofs=[]
    momsprofs=[]
    count=0
    for dump in range(start,end+1,inc):
        num = str(dump).rjust(4,'0')
        if not os.path.exists(runid+num+'.rprof'):
            print('Dump '+num+' does not exist.')
        else:
            file = open(runid+num+'.rprof','rt')
            if dump == start:
                avmomsprofs = getmomsprof(file)
#                print((avmomsprofs[0]))
            else:
                momsprofs = getmomsprof(file)
                for irprof in range(len(momsprofs)):
                    for key in momsprofs[irprof].keys():
                        avmomsprofs[irprof][key]=avmomsprofs[irprof][key]+ \
                            momsprofs[irprof][key]
            count = count + 1      

    for irprof in range(0,len(avmomsprofs)):
        for key in avmomsprofs[irprof].keys():
            avmomsprofs[irprof][key]=avmomsprofs[irprof][key]/count  
    return avmomsprofs

In [93]:
def avrprof(run,dmprange):
    runname=run.split('/')[-1].split('-')[0]
    print('run:',runname)
    os.chdir(run+'/prfs')
    prfs = glob.glob('*.rprof')
    ndumps = len(prfs)
    runid = prfs[0][:-10]
    dump = dmprange[0]
    start,end,inc = dmprange
    count=0
    for dump in range(start,end+1,inc):
        num = str(dump).rjust(4,'0')
        if not os.path.exists(runid+num+'.rprof'):
            print('Dump '+num+' does not exist.')
        else:
            file = open(runid+num+'.rprof','rt')
            if dump == start:
                avparams,avrprofs = getrprof(file)
            else:
                params,rprofs = getrprof(file)
                for key in params.keys():
                    avparams[key]=avparams[key]+params[key]
                for irprof in range(len(rprofs)):
                    for key in rprofs[irprof].keys():
                        avrprofs[irprof][key]=avrprofs[irprof][key]+rprofs[irprof][key] 
            count+=1                 

    for key in avparams.keys():
        avparams[key]=avparams[key]/count
    for irprof in range(len(avrprofs)):
        for key in avrprofs[irprof].keys():
            avrprofs[irprof][key]=avrprofs[irprof][key]/count 
    return [avparams,avrprofs]

In [94]:
def momscompare(runs,ieos,iraddiff,dumps):
    origdir=os.getcwd()
    print(origdir) 
    x,y = [],[]
    xlabel,ylabel=[],[]
    legend,title=[],[]
    sty=style_gen()
    for irun,run in enumerate(runs):
        runname=run.split('/')[-2].split('-')[0]
        print('run:',runname)
        os.chdir(run+'/prfs/frombqavs')
        prfs = glob.glob('*.rprof')
        ndumps = len(prfs)
        runid = prfs[0][:-10]
        for dump in dumps[irun]:
            momsprofs = avmomsprof(run,dump)
            params,rprofs = avrprof(run,dump)
            totallum= params['totallum']
            time=params['time']
            hours=int(time/(60*60))
            idump=str(dump[0])+','+str(dump[1])
#let's just use the first panel of radial profile
            momsprof = momsprofs[0]
            ir = momsprof['ir']
            tot = ir[0]
            r = params['radmax']*ir/tot
            vort=momsprof['vort-sph']
  
            x.append(r)
            xlabel.append(r'$R/\mathrm{Mm}$')    
            y.append(vort)
            ylabel.append(r'$ vort$')
            legend.append(runname+r' $vort$ '+str(hours)+' hrs')      
    return [x,xlabel,y,ylabel,legend] 

In [95]:
sty=style_gen()
path1='/home/user/user/frontera_scratch3_mao1'
runM213moms='/M213-1000xht-1000xdiff-realEOS-768/'
runM207moms='/M207-1000xht-1000xdiff-realEOS-1152/'
runM210moms='/M210-1000xht-1000xdiff-realEOS-1728/'

runM212moms='/M212-31.62xht-31.62xdiff-realEOS-1152/'
runM208moms='/M208-100xht-100xdiff-realEOS-1152/'
runM209moms='/M209-3162.3xht-3162.3xdiff-realEOS-1152/'
runM252moms='/M252-10000x-896-fixKappa/'

runs=[path1+runM212moms,path1+runM208moms,path1+runM207moms,path1+runM209moms,path1+runM252moms]
ieos=[1,1,1]
iraddiff=[1.,100,1000]
#iraddiff=[0,0]
null=[]
a=[[1000,1000,1]]
b=[[1640,1640,1]]
c=[[2000,2000,1]]
d=[[500,500,1]]
dumps=[a,b,c,b,d]

x1,xlabel1,y1,ylabel1,legend1=momscompare(runs,ieos,iraddiff,dumps)
plt.figure()
line_plot(x1[0],np.log10(y1[0]),None,xlabel1[0],ylabel1[0],legend1[0],sty[0])
#plt.hlines(3e-6,0,2500)
line_plot(x1[1],np.log10(y1[1]),None,xlabel1[1],ylabel1[1],legend1[1],sty[1])
#plt.hlines(6e-6,0,2500)
line_plot(x1[2],np.log10(y1[2]),None,xlabel1[2],ylabel1[2],legend1[2],sty[2])
#plt.hlines(1.9e-5,0,2500)
line_plot(x1[3],np.log10(y1[3]),None,xlabel1[3],ylabel1[3],legend1[3],sty[3])
#plt.hlines(2.1e-5,0,2500)
line_plot(x1[4],np.log10(y1[4]),None,xlabel1[4],ylabel1[4],legend1[4],sty[4])
#plt.hlines(2.4e-5,0,2500)
#plt.ylim(0,5e-5)
plt.xlim(1500,2500)
plt.legend()
print(origdir)

/user/dataA_rpod4/Huaqing/ppmstar/projects/H-core-M25/analysis_Prad-runs
run: M212
run: 
run: 
run: M208
run: 
run: 
run: M207
run: 
run: 
run: M209
run: 
run: 
run: M252
run: 
run: 


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/user/dataA_rpod4/Huaqing/ppmstar/projects/H-core-M25/analysis_Prad-runs


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in log10
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log10
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in log10
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in log10
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in log10


In [96]:
plt.close()
plt.figure()
lum=np.array([31.62,100,1000,3162,10000])
vort=np.array([3e-6,6e-6,1.9e-5,2.1e-5,2.4e-5])
plt.plot(np.log10(lum),np.log10(vort))
plt.xlabel('log10 of boosting factor')
plt.ylabel('log10 of vort')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [97]:
sty=style_gen()
path1='/home/user/user/frontera_scratch3_mao1'
path2='/home/user/user/frontera_scratch3_pwoodwar'
runM213moms='/M213-1000xht-1000xdiff-realEOS-768/'
runM207moms='/M207-1000xht-1000xdiff-realEOS-1152/'
runM210moms='/M210-1000xht-1000xdiff-realEOS-1728/'
runM284moms='/M284-2688-Gauss3.5-48MmFWHM-realEOS-1000xhtdiff/'

runs=[path1+runM213moms,path1+runM207moms,path1+runM210moms,path2+runM284moms]
ieos=[1,1,1]
iraddiff=[1000.,1000,1000]
a=[[2000,2000,1]]
b=[[2000,2000,1]]
c=[[2000,2000,1]]
d=[[300,300,1]]
dumps=[a,b,c,d]

x1,xlabel1,y1,ylabel1,legend1=momscompare(runs,ieos,iraddiff,dumps)
plt.figure()
line_plot(x1[0],np.log10(y1[0]),None,xlabel1[0],'log10 '+ylabel1[0],legend1[0],sty[0])
line_plot(x1[1],np.log10(y1[1]),None,xlabel1[1],'log10 '+ylabel1[1],legend1[1],sty[1])
line_plot(x1[2],np.log10(y1[2]),None,xlabel1[2],'log10 '+ylabel1[2],legend1[2],sty[2])
line_plot(x1[3],np.log10(y1[3]),None,xlabel1[3],'log10 '+ylabel1[3],legend1[3],sty[3])
plt.legend(handlelength=5)
print(origdir)
os.chdir('/home/user/user/dataA_rpod4/Huaqing/ppmstar/projects/H-core-M25/analysis_Prad-runs')
plt.savefig('vortRes'+'.pdf',bbox_inches = "tight")

/user/frontera_scratch3_mao1/M252-10000x-896-fixKappa/prfs
run: M213
run: 
run: 
run: M207
run: 
run: 
run: M210
run: 
run: 
run: M284
run: 
run: 


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in log10
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in log10
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: divide by zero encountered in log10
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in log10


/user/dataA_rpod4/Huaqing/ppmstar/projects/H-core-M25/analysis_Prad-runs


In [98]:
def momscompare(runs,ieos,iraddiff,dumps):
    origdir=os.getcwd()
    print(origdir) 
    x,y = [],[]
    xlabel,ylabel=[],[]
    legend,title=[],[]
    sty=style_gen()
    for irun,run in enumerate(runs):
        runname=run.split('/')[-2].split('-')[0]
        print('run:',runname)
        os.chdir(run+'/prfs/frombqavs')
        prfs = glob.glob('*.rprof')
        ndumps = len(prfs)
        runid = prfs[0][:-10]
        for dump in dumps[irun]:
            momsprofs = avmomsprof(run,dump)
            params,rprofs = avrprof(run,dump)
            totallum= params['totallum']
            time=params['time']
            hours=int(time/(60*60))
            idump=str(dump[0])+','+str(dump[1])
#let's just use the first panel of radial profile
            momsprof = momsprofs[0]
            ir = momsprof['ir']
            tot = ir[0]
            r = params['radmax']*ir/tot
            vort=momsprof['vort-sph']
            rho = momsprof['Diverg-U']
            enk = momsprof['fvX1-fv4']
            
            print(hours)
            r,enk=np.flip(r),np.flip(enk)
            dr=abs(r[-1]-r[0])/(len(r)-1.)
            tmp=enk*4.*mt.pi*r**2
            enksum=np.cumsum(tmp*dr)
            r,enk=np.flip(r),np.flip(enk)
            enksum=np.flip(enksum)
            tmp=np.flip(tmp)
  
            x.append(r)
            xlabel.append(r'$R/\mathrm{Mm}$')    
            y.append(enk*rho*4.*np.pi*r**2/(2.*totallum*time))
            ylabel.append(r'$ dE_{kin}/dR$')
            legend.append(runname+r' $dE_{kin}/dR$ '+str(hours)+' hrs')      
    return [x,xlabel,y,ylabel,legend]               

In [99]:
path1='/home/user/user/frontera_scratch3_mao1'
runM201moms='/M201-1000xht-0xdiff-realEOS-1152/M201-moms4ekin'
runM204moms='/M204-1000xht-100xdiff-realEOS-1152/M204-moms4ekin'
runM207moms='/M207-1000xht-1000xdiff-realEOS-1152/M207-moms4ekin'
runs=[path1+runM201moms,path1+runM204moms,path1+runM207moms,]
ieos=[1,1,1]
iraddiff=[1.,100,1000]
#iraddiff=[0,0]
null=[]
a=[[2700,4200,100]]
b=[[2700,4200,100]]
dumps=[a,b,b]
x1,xlabel1,y1,ylabel1,legend1=momscompare(runs,ieos,iraddiff,dumps)
plt.figure()
plt.plot(x1[0],y1[0])
plt.plot(x1[1],y1[1])
plt.plot(x1[2],y1[2])
print(origdir)

/user/dataA_rpod4/Huaqing/ppmstar/projects/H-core-M25/analysis_Prad-runs
run: M201
run: M201
run: M201
2556
run: M204
run: M204
run: M204
2470
run: M207
run: M207
run: M207
2470


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/user/dataA_rpod4/Huaqing/ppmstar/projects/H-core-M25/analysis_Prad-runs


In [100]:
def kappa_func(xh,tlog):
    ffit_par0 = -0.21193353
    ffit_par1 = 4.58822546
    ffit_par2 = -33.25338915
    ffit_par3 = 80.22027956
    kkappa_max = 0.3420244849271527
#    kkappa_max = 0.38
    kkappa_min = 0.32009677883170023
    kkappa_tot_max = 0.66
    llogT_width = 13.0
    llogT0 = 7.06
    cappalog_fit = ffit_par0*tlog + ffit_par1
    cappalog_fit = cappalog_fit*tlog + ffit_par2
    cappalog_fit = cappalog_fit*tlog + ffit_par3
    c_corr = 1.+0.5*(kkappa_max/kkappa_min-1.) \
     *(1.-np.tanh(llogT_width*(tlog-llogT0)))    
    cappa_es = 0.2*(1.+xh)
    cappa = 10.**cappalog_fit*cappa_es/(c_corr*kkappa_min)
    cappa[kkappa_tot_max < cappa]=kkappa_tot_max
    return cappa

def h_fraction(fv,airmu,cldmu,ffkair,ffkcld):
    xh = 0.5*(fv*ffkcld+(1.-fv)*ffkair)/  \
       (fv*cldmu+(1.-fv)*airmu)
    return xh

In [101]:
#compute thermodynamical derivatives that appear in N2 formula (Zahn 1997)
def get_delad(rho,t,mu,a):
#delad=(dlnT/dlnp)_(s,mu)
    t3=t*t*t
    rgas=8.3144617
    rbymu=rgas/mu
    v=1./rho
    delad=(1.5*rbymu+4.*a*v*t3)*rbymu/(rbymu*v+4.*a*t3*v*v/3.)+rbymu*rho+4.*a*t3/3.
    delad=1./delad
    p=rbymu*rho*t+a*t3*t/3.
    delad=delad*p/t
    return delad

def get_delta(rho,t,mu,a):
#delta=-(dlnrho/dlnT)_(p,mu)
    rgas=8.3144617
    rbymu=rgas/mu
    t3=t*t*t
    delta=rbymu*rho+4.*a*t3/3.
    delta=delta/(rbymu*t)
    delta=delta*t/rho
    return delta

def get_phi(rho,t,mu,a):
#phi=(dlnrho/dlnmu)_(p,T)
    phi=1.
    return phi

def get_dpdrho_s(rho,t,mu,a):
#sound speed square
    t4=t*t*t*t
    gm=5./3.
    rgas=8.3144617
    rbymu=rgas/mu
    p=rbymu*rho*t+a*t4/3.
    dpdrho_s=(p+a*t4)**2/(rbymu*rho*t/(gm-1.)+4.*a*t4)
    dpdrho_s=dpdrho_s/rho+rbymu*t
    return dpdrho_s
    
def get_cp(rho,t,mu,a):
#specific heat capacity
    t3=t*t*t
    gm=5./3.
    gm1inv=1./(gm-1.)
    rgas=8.3144617
    rbymu=rgas/mu
    v=1./rho
    p=rbymu*rho*t+a*t3*t/3.
    cp=rbymu*gm1inv+4.*a*t3*v
    cp=cp+(a*t3*t+p)*v*v*(rbymu*rho+4.*a*t3/3.)/(rbymu*t)
    return cp

def get_Hp(p,g,rho,G):
    Hp=-p/(rho*g)
#     for i in range(0,len(p)):
#         if g[i] >= 0:
# #            Hp.append(mt.sqrt(p[i]/G/rho[i]**2))
#             Hp=np.append(Hp,mt.sqrt(p[i]/G/rho[i]**2))
#         else:
# #            Hp.append(min(mt.sqrt(p[i]/G/rho[i]**2),-p[i]/rho[i]/g[i]))
#             Hp=np.append(Hp,min(mt.sqrt(p[i]/G/rho[i]**2),-p[i]/rho[i]/g[i]))   
    return Hp

def get_s(rho,t,mu,a):
    rgas=8.3144617
    rbymu=rgas/mu
    s=-rbymu*np.log(rho)+1.5*rbymu*np.log(t)+4*a*t*t*t/(3.*rho)
    return s
    

In [102]:
def rad_dif(params,rprofs,kradfac):
    rprof = rprofs[0]
    rghi = range(0,len(rprof['IR']))
    rg = range(0,len(rprof['IR']),2)
    rglo = range(0,int(len(rprof['IR'])/2))
    Rhi=rprof['R']
    Rlo=np.array([(Rhi[i] + Rhi[i+1])/2. for i in rg])
    Rho0hi=rprof['Rho0']
    Rho0lo=np.array([(Rho0hi[i] + Rho0hi[i+1])/2. for i in rg])
    tmp=rprof['Rho1']
    Rho1=np.array([(tmp[i] + tmp[i+1])/2. for i in rg])
    Rho=Rho0lo+Rho1
    FVhi=rprof['FV']
    FVlo=np.array([(FVhi[i] + FVhi[i+1])/2. for i in rg])
    tmp=rprof['T9']
    T9=np.array([(tmp[i] + tmp[i+1])/2. for i in rg])
    tlog=np.log10(T9)+9. 
    airmu = params['airmu']
    cldmu = params['cldmu']
    fkair = params['fkair']
    fkcld = params['fkcld']
    Rgasconst = params['Rgasconst']
    if 'radconst' in params:
        radconst = params['radconst']
    else:
        radconst = 756.5767;
    gamma=5./3.
    clight = 299.792458
    dT94=np.gradient(T9**4)
    drad=np.gradient(Rlo)
    X=h_fraction(FVlo,airmu,cldmu,fkair,fkcld)
    muav= FVlo*cldmu+(1.-FVlo)*airmu
    tlog = np.log10(T9*10**9)
    cappa=kappa_func(X,tlog)
#     cappa=kappa_func28(xh,tlog,Rho)
    flx = -clight * radconst*dT94
    denom = 3.* cappa * drad * Rho
    flx = flx / denom
    radflx=4.*mt.pi*flx*Rlo**2*0.00000000001*kradfac
#compute cp specific heat capacity
    at3byrho=radconst*T9*T9*T9/Rho
#        cpav=Rgasconst/((gamma-1.)*muav)+(5.+4.*muav*at3byrho/(3.*Rgasconst))\
#        *at3byrho
    cpav= get_cp(Rho,T9,muav,radconst)
    rad=Rlo

    return  T9,cappa,rad,radflx

path1='/home/user/user/frontera_scratch3_mao1'
path2='/home/user/user/niagara_scratch_fherwig'
runM207='/M207-1000xht-1000xdiff-realEOS-1152'
runV4='/V4-100xL-100xK-1152-TyroneM1e4_0.1'
runM252='/M252-10000x-896'
runpath = path1 + runM207
os.chdir(runpath+'/prfs/')
prfs = glob.glob('*.rprof')
ndumps = len(prfs)
dumps = [int(prfs[i][12:16]) for i in range(0,ndumps)]
dumps.sort()
nbegin = dumps[0]
nend = dumps[-1]
runid = prfs[0][:-10]
plt.close()
plt.grid()
plt.figure()
for nnum in[0,]:
    num = str(nnum).rjust(4,'0')
    file = open(runid+num+'.rprof','rt')
    params,rprofs = getrprof(file)
    tlog,cappa,r,radflx = rad_dif(params,rprofs,1000)
    plt.xlabel('radius')
    line_plot(r,cappa,None,'R','kappa ','dump:'+str(num),[':','*','b'])
plt.legend(loc='lower right',prop={'size': 8})
os.chdir(origdir)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [103]:
def compare(runs,ieos,iraddiff,dumps):
    origdir=os.getcwd()
    print(origdir) 
    x,y = [],[]
    xlabel,ylabel = [],[]
    legend,title=[],[]
    sty=style_gen()
    for irun,run in enumerate(runs):
        runname=run.split('/')[-1].split('-')[0]
        print('run:',runname)
        os.chdir(run+'/prfs')
        prfs = glob.glob('*.rprof')
        ndumps = len(prfs)
        runid = prfs[0][:-10]
        alldumps = [int(prfs[i][12:16]) for i in range(0,ndumps)]
        alldumps.sort()
#### read in T90 separately
        num = str(0).rjust(4,'0')
        file = open(runid+num+'.rprof','rt')
        params,rprofs = getrprof(file)
        rprof=rprofs[0]
        IRhi = rprof['IR']
        rg = range(0,len(rprof['IR']),2)
        T90hi = rprof['T9']
        T90lo = np.array([(T90hi[i]+T90hi[i+1])/2. for i in rg])
        Rho0hi = rprof['Rho0']
        Rho0lo = np.array([(Rho0hi[i]+Rho0hi[i+1])/2. for i in rg])
        FV0hi = rprof['FV']
        FV0lo = np.array([(FV0hi[i]+FV0hi[i+1])/2. for i in rg])
        airmu = params['airmu']
        cldmu = params['cldmu']
        if 'radconst' in params:
            radconst = params['radconst']
        else:
            radconst = 756.5767;
        if ieos[irun]==0:
            radconst = 0.
        Rgasconst = params['Rgasconst']
        amu=FV0lo*cldmu+(1.-FV0lo)*airmu
        eint0lo=Rgasconst*T90lo/(amu*2./3.)+radconst*T90lo**4/Rho0lo
###
        totcldms=[]
        totms=[]
        t=[]
        for dump in dumps[irun]:
            params,rprofs = avrprof(run,dump)
            idump=str(dump[0])+','+str(dump[1])
            dx = params['dx']
            time = params['time']
            hours=int(time/(60*60))
            print('time:',hours,' hours')
            radout0 = params['radout0']
            radmax = params['radmax']
            totallum = params['totallum']
            radin0 = params['radin0']
            g=params['gravconst']
            t.insert(0,time)
#let's just use the first panel of radial profile
            rprof = rprofs[0]
            rghi = range(0,len(rprof['IR']))
            rg = range(0,len(rprof['IR']),2)
            IRhi = rprof['IR']
            Rhi = rprof['R']
            Rlo = np.array([(Rhi[i]+Rhi[i+1])/2. for i in rg])
            Rho0hi = rprof['Rho0']
            Rho0lo = np.array([(Rho0hi[i]+Rho0hi[i+1])/2. for i in rg])
            rglo = range(0,len(Rho0lo))
            tmp = rprof['Rho1']
            Rho1=np.array([(tmp[i]+tmp[i+1])/2. for i in rg])
            P0hi = rprof['P0']
            P0lo = np.array([(P0hi[i]+P0hi[i+1])/2. for i in rg])
                
            tmp = rprof['P1']
            P1=np.array([(tmp[i]+tmp[i+1])/2. for i in rg])
            FVhi = rprof['FV']
            FVlo=np.array([(FVhi[i]+FVhi[i+1])/2. for i in rg])
            dFVdRlo = np.gradient(FVlo,Rlo)
            tmp = rprof['T9']
            T9=np.array([(tmp[i]+tmp[i+1])/2. for i in rg])
            airmu = params['airmu']
            cldmu = params['cldmu']
            muhi =airmu*(1.-FVhi)+cldmu*FVhi
            mulo=np.array([(muhi[i]+muhi[i+1])/2. for i in rg])
            fkair = params['fkair']
            fkcld = params['fkcld']
            if 'radconst' in params:
                radconst = params['radconst']
            else:
                radconst = 756.5767;
            if ieos[irun] == 0:
                radconst=0.
            Rgasconst = params['Rgasconst']
            Rho =Rho0lo+Rho1
            P =P0lo+P1
            s=get_s(Rho,T9,mulo,radconst) 
            
            Prad =(radconst*T9**4)/3.
            
            tmp=np.gradient(P0hi,Rhi)
#gravitational acceleration and internal energy per unit mass
            g0hi=tmp/Rho0hi
            g0lo=np.array([(g0hi[i]+g0hi[i+1])/2. for i in rg])
            amu=FVlo*cldmu+(1.-FVlo)*airmu
            eint=Rgasconst*T9/(amu*2./3.)+radconst*T9**4/Rho
                    
            delta=get_delta(Rho,T9,mulo,radconst) 
            delad=get_delad(Rho,T9,mulo,radconst) 
#                Hp0lo=[abs(P0lo[i]/max(-Rho0lo[i]*g0lo[i],1e-6)) for i in rglo]
            Hp0lo=get_Hp(P0lo,g0lo,Rho0lo,g)
            phi=get_phi(Rho,T9,mulo,radconst) 
            inx=np.array([i for i in rglo])
            dtdi=np.gradient(T9,inx)
            dpdi=np.gradient(P,inx)
            drhodi=np.gradient(Rho,inx)
            dpdr=np.gradient(P,Rlo)
            dmudilo=np.gradient(mulo,inx)
            dtdi[dtdi==0]=10*-10
            dpdi[dpdi==0]=10*-10
#dell is dlnT/dlnp of the star stratification
            dell=P*dtdi/(T9*np.maximum(10*-12,dpdi))
            dlnp=dpdi/P
            dlnrho=drhodi/Rho
#                Hp=[abs(P[j]/max(1e-6,-dpdr[j])) for j in rglo]
            Hp=get_Hp(P,g0lo,Rho,g)
            dlnmulo=dmudilo/mulo
            Ntsq=g0lo**2*Rho*delta*(delad-dell)/P
            Nmusq=g0lo**2*phi*Rho*dmudilo/(dpdi*mulo)
            
            sig=300.
            sqrt2=mt.sqrt(2)
            sqrtpi=mt.sqrt(mt.pi)
            LNlo=totallum*( erf(Rlo/(sig*sqrt2)) - np.exp(-(Rlo/sig)**2/2.)*sqrt2*Rlo/(sig*sqrtpi) ) 
            
# compute potential as a function of radius            
            Rhi=np.flip(Rhi)
            g0hi=np.flip(g0hi)
            dRhi = np.gradient(Rhi)
            pot0hi = np.cumsum(-g0hi*dRhi)
            Rhi=np.flip(Rhi)
            g0hi=np.flip(g0hi)
            pot0hi=np.flip(pot0hi)                                          
####                                  
            rprof = rprofs[1]
            offset = 1 if 'Ceul' not in rprof else 0 
            rprof = rprofs[1+offset]
            rglo = range(0,len(rprof['IR']))
            IRlo = rprof['IR']
            Rlo = rprof['R']
            Ceul=rprof['Ceul']
            Uabs=rprof['|U|']
            A=rprof['A']
            isflat=0
            m,cldm=get_mass(Rlo,Rho,FVlo,airmu,cldmu,isflat)
            m1,tmp=get_mass(Rlo,Rho1,FVlo,airmu,cldmu,isflat)
# 1 unit of mass = 10**27 gm
            m0lo,tmp=get_mass(Rlo,Rho0lo,FVlo,airmu,cldmu,isflat)
            totcldms.append(cldm[0])
            totms.append(m[0])
            glo = g*m/Rlo/Rlo
    
# compute potential as a function of radius            
            Rlo=np.flip(Rlo)
            g0lo=np.flip(g0lo)
            dRlo = np.gradient(Rlo)
            pot0lo = np.cumsum(-g0lo*dRlo)
            Rlo=np.flip(Rlo)
            g0lo=np.flip(g0lo)
            pot0lo=np.flip(pot0lo) 
                
#compute potential and its perturbation in each radial shell
            Rlo=np.flip(Rlo)
            Rho1=np.flip(Rho1)
            Rho0lo=np.flip(Rho0lo)
            pot0lo=np.flip(pot0lo)
            dV = np.gradient(4.*mt.pi*Rlo**3./3.)
            Epot1 = np.cumsum(pot0lo*Rho1*dV) 
            Epot0lo = np.cumsum(pot0lo*Rho0lo*dV)  
            Rlo=np.flip(Rlo)
            Rho1=np.flip(Rho1)
            Rho0lo=np.flip(Rho0lo)
            pot0lo=np.flip(pot0lo)
            Epot1=np.flip(Epot1)
            Epot0lo=np.flip(Epot0lo)
        
            dPdR = np.gradient(P,Rlo)
            dAdR = np.gradient(A,Rlo)
            dRhodR = np.gradient(Rho,Rlo)
            Nsq=np.maximum(0,(dRhodR-dPdR/(Ceul*Ceul))\
                *dPdR/(Rho*Rho)) 
                
            rprof = rprofs[2+offset]
            RhoUr=rprof['RhoUr']
            RhoUrUsq=rprof['RhoUrUsq']
            RhoUrH=rprof['RhoUrH']
            flxkin=RhoUrUsq*4.*mt.pi*Rlo**2 
            tmp=RhoUrH*4.*mt.pi*Rlo**2 
            F_enth=smooth(tmp,5)
                
            rprof = rprofs[3+offset]
            Ut=rprof['|Ut|']
            dUtdR=np.gradient(Ut,Rlo)
            Ur=rprof['Ur']
            dUrRMS=rprof['dUrRMS']
            dUtRMS =rprof['d|Ut|RMS']     

            enint1=Rho*eint-Rho0lo*eint0lo
#this is not kinetic energy! just some very inaccurate estimate!
            enk=Rho*Uabs**2./2.
            entot1=enint1+enk              
             
            tmp1=smooth(P,2)
            tmp2=smooth(T9,2)
            dTdp_star=np.gradient(tmp2,tmp1)
            dlnTdlnp_star=dTdp_star*tmp1/tmp2
                    
#def get_delad(rho,t,mu,a):
#delad=(dlnT/dlnp)_(s,mu)
            amu=FVlo*cldmu+(1.-FVlo)*airmu
            dlnTdlnp_ad=get_delad(Rho,T9,amu,radconst)
            cp=get_cp(Rho,T9,amu,radconst)
                
#def get_delta(rho,t,mu,a):
#def get_phi(rho,t,mu,a):
            delta=get_delta(Rho,T9,amu,radconst)
            phi=get_phi(Rho,T9,amu,radconst)

            dmudp_star=np.gradient(amu,P)
            dlnmudlnp_star=dmudp_star*P/amu                    
                
            dP0dRhi = np.gradient(P0hi,Rhi)
            g0hi=dP0dRhi/Rho0hi
            g0lo=[(g0hi[i]+g0hi[i+1])/2. for i in rg]
            isflat=0
            m,cldm=get_mass(Rlo,Rho,FVlo,airmu,cldmu,isflat)
            m1,tmp=get_mass(Rlo,Rho1,FVlo,airmu,cldmu,isflat)
            m0lo,cldm0lo=get_mass(Rlo,Rho0lo,FV0lo,airmu,cldmu,isflat)
#               g0lo=[-m0lo[j]*6.67*(10**-5)/Rlo[j]**2 for j in rglo]
            drlo=abs(Rlo[0]-Rlo[-1])/(len(Rlo)-1)
            entolift=g0lo*m1*drlo
###               
            entoliftsum=[]
            entolift=np.flip(entolift)
            entoliftsum = np.cumsum(entolift)                        
            entolift=np.flip(entolift)
            entoliftsum=np.flip(entoliftsum)
###               
    
            gamma=5./3.
# thermal conductivity and thermal diffusivity
            clight=299.792458
            X=h_fraction(FVlo,airmu,cldmu,fkair,fkcld)
            muav= FVlo*cldmu+(1.-FVlo)*airmu
            tlog = np.log10(T9*10**9)
            cappa=kappa_func(X,tlog)
#                cappaav=kappa_func28(X,thyng,Rho[i])
#compute cp specific heat capacity
            cp=get_cp(Rho,T9,muav,radconst)             

###
            tmp=-RhoUr*g0lo*drlo*4.*mt.pi*Rlo**2 
            tmp=np.flip(tmp)
            potflx=np.cumsum(tmp)
            potflx=np.flip(potflx)
###
            
            Pgrav=RhoUr*g0lo*4.*mt.pi*Rlo*Rlo
            Pgrav=np.convolve(Pgrav,[1./5.]*5,mode='same')
        
###             
            tmp=enint1*drlo*4.*mt.pi*Rlo**2
            tmp=np.flip(tmp)
            enint1sum=np.cumsum(tmp)
            enint1sum=np.flip(enint1sum)
###
                
            c=299.792458 
            tout,cappa,r,radflx = rad_dif(params,rprofs,iraddiff[irun])
# F_rad should be the same with radflx
            F_rad=(16.*mt.pi*radconst*g*c*m*T9**4*iraddiff[irun]*dlnTdlnp_star) \
                      /(10**11*3.*cappa*P)
            nabla_rad=10**11*3.*cappa*(F_enth+F_rad)*P\
                        /(16.*mt.pi*radconst*g*c*m*T9**4*iraddiff[irun]) 
            delrad=10**11*3.*cappa*(F_enth+F_rad)*P\
                        /(16.*mt.pi*radconst*g*c*m*T9**4*iraddiff[irun]) 

            tmp=(dlnTdlnp_star-dlnTdlnp_ad) 
            supad=np.convolve(tmp, [0.2]*5, mode='same')
                
            tmp=4.*mt.pi*Rlo**2*Rho*cp*T9*np.sqrt(P/Rho)*np.maximum(supad,0)**1.5 
            asq=F_enth/tmp
            
#thermal diffusivity                
            D=(1e-11)*iraddiff[irun]*4.*radconst*clight*T9**3 \
                /(3*cappa*cp*Rho**2) 
                
            lph=iraddiff[irun]*(1e-11)/(cappa*Rho)
                
#            Fconvbylmsq=[Rho[i]*cp[i]*T9[i]*mt.sqrt(abs(g0lo[i])*delta[i]) \
#                            *Hp[i]**(-1.5)*(dlnTdlnp_star[i]-0.999*dlnTdlnp_ad[i])**1.5/(4.*mt.sqrt(2)) for i in rglo]
#            Fconvbylmsq=smooth(Fconvbylmsq,5)
                
#            asq=[RhoUrH[i]/Fconvbylmsq[i]/Hp[i]/Hp[i] for i in rglo]
#            asq=smooth(asq,5)

            Fconvbylmsq=Rho*cp*T9*np.sqrt(np.absolute(g0lo)*delta) \
                            *Hp**(-1.5)*(dlnTdlnp_star-0.999*dlnTdlnp_ad)**1.5/(4.*np.sqrt(2)) 
            Fconvbylmsq=smooth(Fconvbylmsq,5)
                
            asq=RhoUrH/Fconvbylmsq/Hp/Hp
            asq=smooth(asq,5)
#1st
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$')    
            y.append(asq)
            ylabel.append(r'$ \alpha ^2$')
            legend.append(runname+r' $\alpha ^2$ '+str(hours)+' hrs')
                
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$')    
            y.append(Prad/P)
            ylabel.append(r'$ Prad/P $')
            legend.append(runname+r' $\Prad/P $ '+str(hours)+' hrs')
                
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$')    
            y.append(cp)
            ylabel.append(r'$ cp $')
            legend.append(runname+r' $cp $ '+str(hours)+' hrs')
                
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$')    
            y.append(A)
            ylabel.append(r' entropy')
            legend.append(runname+r' $A$ '+str(hours)+' hrs')
                
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$')    
            y.append(dFVdRlo )
            ylabel.append(r' grad FV')
            legend.append(runname+r' $grad FV$ '+str(hours)+' hrs')
                
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$')    
            y.append(dAdR)
            ylabel.append(r' grad entropy ')
            legend.append(runname+r' $dAdR$ '+str(hours)+' hrs')
                
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$')    
            y.append(Uabs)
            ylabel.append(r' Uabs / Mm/s')
            legend.append(runname+r' $Uabs$ '+str(hours)+' hrs')
                
            x.append(Rlo.copy())
#            x.append(m.copy()/Msun)
            xlabel.append(r'$R/\mathrm{Mm}$')    
#            xlabel.append(r' $m/M_{sun}$') 
            y.append(FVlo)
            ylabel.append(r' FV ')
            legend.append(runname+r' $FV$ '+str(hours)+' hrs')
                
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$')    
            y.append(Ut)
            ylabel.append(r' Ut / Mm/s')
            legend.append(runname+r' $Ut$ '+str(hours)+' hrs')
                
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$')    
            y.append(Ur)
            ylabel.append(r' Ur / Mm/s')
            legend.append(runname+r' $Ur$ '+str(hours)+' hrs')

#11th
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$')    
            y.append(RhoUr)
            ylabel.append(r' RhoUr')
            legend.append(runname+r' $RhoUr$ '+str(hours)+' hrs')
                
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$')    
            y.append(delta)
            ylabel.append(r' delta')
            legend.append(runname+r' $delta$ '+str(hours)+' hrs')
                
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$')    
            y.append(P0lo)
            ylabel.append(r' P0lo')
            legend.append(runname+r' $P0lo$ '+str(hours)+' hrs')
                
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$')    
            y.append(Rho0lo)
            ylabel.append(r' Rho0lo')
            legend.append(runname+r' $Rho0lo$ '+str(hours)+' hrs')
                
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$')    
            y.append(P1)
            ylabel.append(r' P1')
            legend.append(runname+r' $P1$ '+str(hours)+' hrs')
                
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$')    
            y.append(Rho1/Rho0lo)
            ylabel.append(r'$\rho f$')
            legend.append(runname+r' $\rho f$ '+str(hours)+' hrs')

            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$')
#             y.append(Epot1/(totallum*np.maximum(time,10**-6)) )
            y.append((pot0lo*Rho1*np.pi*4.*Rlo*Rlo)/(totallum*np.maximum(time,10**-6)) )
            ylabel.append(r' $energy  / (\mathrm{L \times time}$)')
            legend.append(runname+' $dE^{pot}_1/dR$ '+str(hours)+' hrs')
                
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$')    
            y.append(enint1*4.*mt.pi*Rlo*Rlo/(totallum*np.maximum(time,10**-6)) )
            ylabel.append(r' $energy  / (\mathrm{L \times time}$)')
            legend.append(runname+' $dE^{int}_1/dR$ '+str(hours)+' hrs')
                
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$')
            y.append(Pgrav/totallum )
            ylabel.append(r' \frac{dP}{dR}/ L')
            legend.append(runname+' '+r'$4\pi R^2 \rho U_R g$')
                
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$')
            y.append(flxkin/totallum)
            ylabel.append(r' power  / L')
            legend.append(runname+' $F^{\mathrm{kin}}$ '+str(hours)+' hrs')

#21st
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$')
            y.append(F_enth/totallum)
            ylabel.append(r' power  / L')
            legend.append(runname+' $F_{\mathrm{conv}}$')
                
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$')
            radflx=np.convolve(radflx, [1./20.]*20, mode='same')
            y.append(radflx/totallum)
            ylabel.append(r' power  / L')
            legend.append(runname+' $F_{\mathrm{rad}}$')
            
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$')
            y.append((radflx+F_enth)/totallum )
            ylabel.append(r' power  / L')
            legend.append(runname+' $F_{\mathrm{tot}}$')
                
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$')
            y.append(cappa)
            ylabel.append(r' cappa')
            legend.append(runname+' $cappa$ '+str(hours)+' hrs')
                
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$')    
            y.append(RhoUrH)
            ylabel.append(r' power')
            legend.append(runname+' RhoUrH '+str(hours)+' hrs')
                          
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$')    
            y.append(dlnTdlnp_star)
            ylabel.append(r'derivative')
            legend.append(runname+r' $\nabla_{star}$ '+str(hours)+' hrs')
                
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$')
            y.append(nabla_rad)
            ylabel.append(r' derivative')
            legend.append(runname+r' $\nabla_{rad}$ '+str(hours)+' hrs')
                
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$')    
            y.append(dlnTdlnp_ad)
            ylabel.append(r' derivative')
            legend.append(runname+r' $\nabla_{ad}$ '+str(hours)+' hrs')
                
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$')   
            y.append(supad)
            ylabel.append(r' derivative')
            legend.append(runname+' superadiabat '+str(hours)+' hrs')
                
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$')   
            tmp=(dlnTdlnp_ad-dlnTdlnp_star+phi*dlnmudlnp_star/delta)
            y.append(tmp)
            ylabel.append(r' derivative')
            legend.append(runname+' Ledoux: '+str(hours)+' hrs')

#31st
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$')   
            y.append(supad/dlnTdlnp_ad)
            ylabel.append(r' derivative')
            legend.append(runname+' supad fraction: '+str(hours)+' hrs')
                
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$') 
            y.append(asq)
#                ylabel.append(r' $\alpha^2$')
#                legend.append(runname+r' $\alpha^2$ dump: '+str(idump))
            ylabel.append(r' $\alpha^2 $')
            legend.append(runname+r' $\alpha^2 $ '+str(hours)+' hrs')
                
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$') 
            y.append(Nsq)
            ylabel.append(r' $N^2/Hz^2$')
            legend.append(runname+r' $N^2$ '+str(hours)+' hrs')
                
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$') 
            y.append(Ntsq)
            ylabel.append(r' $N^2/Hz^2$')
            legend.append(runname+r' $N_t^2$ '+str(hours)+' hrs')
                
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$') 
            y.append(Nmusq)
            ylabel.append(r' $N^2/Hz^2$')
            legend.append(runname+r' $N_{\mu}^2$ '+str(hours)+' hrs')
                
            Ulmsq=(1e-11)*iraddiff[irun]*3.*radconst*clight*T9**3 * np.sqrt(abs(-8*Hp/(np.minimum(g0lo,-1e-6)*delta)))/(cp*Rho**2*cappa)
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$')    
            y.append(Ulmsq)
            ylabel.append(r' Ulmsq')
            legend.append(runname+r' Ulmsq '+str(hours)+' hrs')
                
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$') 
            y.append(Hp)
            ylabel.append(r' $Hp0lo$')
            legend.append(runname+r' $Hp$ '+str(hours)+' hrs')
                
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$') 
            y.append(g0lo)
            ylabel.append(r' $g0lo$')
            legend.append(runname+r' $g0lo$ '+str(hours)+' hrs')
                
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$') 
            y.append(m/Msun)
            ylabel.append(r' $m/M_{sun}$')
            legend.append(runname+r' $m/M_{sun}$ '+str(hours)+' hrs')
                
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$') 
            y.append(-glo)
            ylabel.append(r' $g$')
            legend.append(runname+r' $g$ '+str(hours)+' hrs')

#41st
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$') 
            y.append(lph)
            ylabel.append(r' $lph$')
            legend.append(runname+r' $lph$ '+str(hours)+' hrs')
                
#MLT                              
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$') 
            y.append(enint1/(eint0lo*Rho0lo))
            ylabel.append(r' $\epsilon f$')
            legend.append(runname+r' $\epsilon f$ '+str(hours)+' hrs')
                
            x.append(Rlo.copy())
            xlabel.append(r'$R/\mathrm{Mm}$') 
            tmp=cp*Rho*T9*np.sqrt(P/Rho)*(np.maximum(0,dlnTdlnp_star-dlnTdlnp_ad))**1.5
            a=[]
            for i in rglo:
                if tmp[i] == 0:
                    a.append(0)
                else:
                    a.append(RhoUrH[i]/tmp[i])
            a=np.convolve(a, [1./5.]*5, mode='same')
            y.append(a)
            ylabel.append(r' $\alpha^2$ dereived from flux')
            legend.append(runname+r' $\alpha^2$ derived from flux '+str(hours)+' hrs')
                       
    return [x,xlabel,y,ylabel,legend]               

Compare M201,M204,M207

In [104]:
path1='/home/user/user/frontera_scratch3_mao1'
path2='/home/user/user/niagara_scratch_fherwig'
runM207='/M207-1000xht-1000xdiff-realEOS-1152'
runM204='/M204-1000xht-100xdiff-realEOS-1152'
runM201='/M201-1000xht-0xdiff-realEOS-1152'
runM215='/M215-shell-ht-768'
runM208='/M208-100xht-100xdiff-realEOS-1152'
runM212='/M212-31.62xht-31.62xdiff-realEOS-1152'
runM209='/M209-3162.3xht-3162.3xdiff-realEOS-1152'
runM114='/M114'
runs=[path1+runM201,path1+runM204,path1+runM207]
#runs=[path1+runM201,]
ieos=[1,1,1,]
iraddiff=[1,100,1000,]
#iraddiff=[0,0]
a=[[2700,3960,50]]
b=[[2700,4200,50]]
dumps=[a,b,b]

In [105]:
print(dumps[0])
x,xlabel,y,ylabel,legend=compare(runs,ieos,iraddiff,dumps)

[[2700, 3960, 50]]
/user/dataA_rpod4/Huaqing/ppmstar/projects/H-core-M25/analysis_Prad-runs
run: M201
run: M201
time: 2467  hours
run: M204


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:290: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:306: RuntimeWarning: invalid value encountered in power


run: M204
time: 2470  hours
run: M207
run: M207
time: 2470  hours


In [106]:
for i in range(0,43):
    print(i,xlabel[i],ylabel[i],str(legend[i]))
print(origdir)

0 $R/\mathrm{Mm}$ $ \alpha ^2$ M201 $\alpha ^2$ 2467 hrs
1 $R/\mathrm{Mm}$ $ Prad/P $ M201 $\Prad/P $ 2467 hrs
2 $R/\mathrm{Mm}$ $ cp $ M201 $cp $ 2467 hrs
3 $R/\mathrm{Mm}$  entropy M201 $A$ 2467 hrs
4 $R/\mathrm{Mm}$  grad FV M201 $grad FV$ 2467 hrs
5 $R/\mathrm{Mm}$  grad entropy  M201 $dAdR$ 2467 hrs
6 $R/\mathrm{Mm}$  Uabs / Mm/s M201 $Uabs$ 2467 hrs
7 $R/\mathrm{Mm}$  FV  M201 $FV$ 2467 hrs
8 $R/\mathrm{Mm}$  Ut / Mm/s M201 $Ut$ 2467 hrs
9 $R/\mathrm{Mm}$  Ur / Mm/s M201 $Ur$ 2467 hrs
10 $R/\mathrm{Mm}$  RhoUr M201 $RhoUr$ 2467 hrs
11 $R/\mathrm{Mm}$  delta M201 $delta$ 2467 hrs
12 $R/\mathrm{Mm}$  P0lo M201 $P0lo$ 2467 hrs
13 $R/\mathrm{Mm}$  Rho0lo M201 $Rho0lo$ 2467 hrs
14 $R/\mathrm{Mm}$  P1 M201 $P1$ 2467 hrs
15 $R/\mathrm{Mm}$ $\rho f$ M201 $\rho f$ 2467 hrs
16 $R/\mathrm{Mm}$  $energy  / (\mathrm{L \times time}$) M201 $dE^{pot}_1/dR$ 2467 hrs
17 $R/\mathrm{Mm}$  $energy  / (\mathrm{L \times time}$) M201 $dE^{int}_1/dR$ 2467 hrs
18 $R/\mathrm{Mm}$  \frac{dP}{dR}/ L M201 $4\

Plot cp and Prad/P

In [107]:
sty=style_gen()
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ln1=ax1.plot(x[1], y[1],label=legend[1],markersize=4,markevery=30,\
         linestyle=sty[0][0], marker=sty[0][1], color=sty[0][2])
ln2=ax2.plot(x[2], y[2],label=legend[2],markersize=4,markevery=30,\
         linestyle=sty[1][0], marker=sty[1][1], color=sty[1][2])
ln3=ax1.plot(x[44], y[44],label=legend[44],markersize=4,markevery=30,\
         linestyle=sty[2][0], marker=sty[2][1], color=sty[2][2])
ln4=ax2.plot(x[45], y[45],label=legend[45],markersize=4,markevery=30,\
         linestyle=sty[3][0], marker=sty[3][1], color=sty[3][2])
lns = ln1+ln2+ln3+ln4
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs, loc=0)
ax1.set_xlabel(r"$R/Mm$")
ax1.set_ylabel(r"$P_{rad}/P$")
ax2.set_ylabel(r"$c_p$")
os.chdir(origdir)
plt.savefig('Pradf'+'.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [108]:
sty=style_gen()
fig, ax1 = plt.subplots()
fig.set_tight_layout(True)
ax2=ax1.twinx()
plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
ln1=ax1.plot(x[16], y[16],label=legend[16],markersize=4,markevery=30,\
         linestyle=sty[0][0], marker=sty[0][1], color=sty[0][2])
ln2=ax1.plot(x[17], y[17],label=legend[17],markersize=4,markevery=30,\
         linestyle=sty[1][0], marker=sty[1][1], color=sty[1][2])
ln3=ax1.plot(x[59], y[59],label=legend[59],markersize=4,markevery=30,\
         linestyle=sty[2][0], marker=sty[2][1], color=sty[2][2])
ln4=ax1.plot(x[60], y[60],label=legend[60],markersize=4,markevery=30,\
         linestyle=sty[3][0], marker=sty[3][1], color=sty[3][2])
ln5=ax1.plot(x[102], y[102],label=legend[102],markersize=4,markevery=30,\
         linestyle=sty[4][0], marker=sty[4][1], color=sty[4][2])
ln6=ax1.plot(x[103], y[103],label=legend[103],markersize=4,markevery=30,\
         linestyle=sty[5][0], marker=sty[5][1], color=sty[5][2])

ln7=ax2.plot(x1[0], y1[0],label=legend1[0],markersize=4,markevery=30,\
         linestyle=sty[5][0], marker=sty[6][1], color=sty[6][2])
ln8=ax2.plot(x1[1], y1[1],label=legend1[1],markersize=4,markevery=30,\
         linestyle=sty[5][0], marker=sty[7][1], color=sty[7][2])
ln9=ax2.plot(x1[2], y1[2],label=legend1[2],markersize=4,markevery=30,\
         linestyle=sty[5][0], marker=sty[8][1], color=sty[8][2])

lns = ln1+ln2+ln3+ln4+ln5+ln6+ln7+ln8+ln9
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs, loc=0,prop={'size': 8},handlelength=4)
ax1.set_xlabel(r"$R/Mm$")
ax1.set_ylabel(r" $\frac{dE^{pot,int}_1}{dR}  / (\mathrm{L \times time}$)")
ax2.set_ylabel(r" $\frac{dE^{kin}}{dR}  / (\mathrm{L \times time}$)")

os.chdir(origdir)
print(origdir)
plt.savefig('1000xht-energy'+'.pdf',bbox_inches = "tight")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/user/dataA_rpod4/Huaqing/ppmstar/projects/H-core-M25/analysis_Prad-runs


In [109]:
sty=style_gen()
fig, ax1 = plt.subplots()
fig.set_tight_layout(True)
ln1=ax1.plot(x[18], y[18],label=legend[18],markersize=4,markevery=30,\
         linestyle=sty[0][0], marker=sty[0][1], color=sty[0][2])
ln2=ax1.plot(x[61], y[61],label=legend[61],markersize=4,markevery=30,\
         linestyle=sty[0][0], marker=sty[0][1], color=sty[1][2])
ln3=ax1.plot(x[104], y[104],label=legend[104],markersize=4,markevery=30,\
         linestyle=sty[0][0], marker=sty[0][1], color=sty[2][2])

lns = ln1+ln2+ln3
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs, loc=0,prop={'size': 8},handlelength=4)
ax1.set_xlabel(r"$R/Mm$")
ax1.set_ylabel(r" $\frac{dP}{dR}  / L $")
os.chdir(origdir)
print(origdir)
plt.savefig('1000xht-Pgrav'+'.pdf',bbox_inches = "tight")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/user/dataA_rpod4/Huaqing/ppmstar/projects/H-core-M25/analysis_Prad-runs


In [110]:
sty=style_gen()
fig, ax1 = plt.subplots()
fig.set_tight_layout(True)
ax2 = ax1.twinx()
ln1=ax1.plot(x[15], y[15],label=legend[15],markersize=4,markevery=30,\
         linestyle=sty[0][0], marker=sty[0][1], color=sty[0][2])
ln2=ax1.plot(x[41], y[41],label=legend[41],markersize=4,markevery=30,\
         linestyle=sty[1][0], marker=sty[1][1], color=sty[1][2])
ln3=ax1.plot(x[58], y[58],label=legend[58],markersize=4,markevery=30,\
         linestyle=sty[2][0], marker=sty[2][1], color=sty[2][2])
ln4=ax1.plot(x[84], y[84],label=legend[84],markersize=4,markevery=30,\
         linestyle=sty[3][0], marker=sty[3][1], color=sty[3][2])
ln5=ax1.plot(x[101], y[101],label=legend[101],markersize=4,markevery=30,\
         linestyle=sty[4][0], marker=sty[4][1], color=sty[4][2])
ln6=ax1.plot(x[127], y[127],label=legend[127],markersize=4,markevery=30,\
         linestyle=sty[5][0], marker=sty[5][1], color=sty[5][2])

ln7=ax2.plot(x[6], y[6],label=legend[6],markersize=4,markevery=30,\
         linestyle=sty[6][0], marker=sty[6][1], color=sty[6][2])
ln8=ax2.plot(x[49], y[49],label=legend[49],markersize=4,markevery=30,\
         linestyle=sty[7][0], marker=sty[7][1], color=sty[7][2])
ln9=ax2.plot(x[92], y[92],label=legend[92],markersize=4,markevery=30,\
         linestyle=sty[8][0], marker=sty[8][1], color=sty[8][2])
lns = ln1+ln2+ln3+ln4+ln5+ln6+ln7+ln8+ln9
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs, loc=0,prop={'size': 8},handlelength=4)
ax1.set_xlabel(r"$R/Mm$")
ax1.set_ylabel(r"perturbation")
ax2.set_ylabel(r"|U|")
os.chdir(origdir)
print(origdir)
plt.savefig('1000xht-pert'+'.pdf',bbox_inches = "tight")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/user/dataA_rpod4/Huaqing/ppmstar/projects/H-core-M25/analysis_Prad-runs


In [111]:
plt.close()
sty=style_gen()
plt.figure(dpi=200)
plt.grid()
#for i,ix in enumerate([27,68,109]):
#for i,ix in enumerate([0,41,82]):
a=[20,21,22]
nvar=43
num= len(a)
for i in range(0,num):
    a.append(a[i]+nvar)
    a.append(a[i]+nvar*2)
for i,ix in enumerate(a,):
    titl=None
#        plt.yscale('log')
#    plt.xlim(0,2450)
    plt.ylim(-0.3,1.3)
    line_plot(x[ix],y[ix],titl,xlabel[ix],ylabel[ix],legend[ix],sty[i])

os.chdir(origdir)
#plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
#plt.legend(bbox_to_anchor=(1.,1),loc='upper left',fontsize=6)
plt.legend(loc='upper left',prop={'size': 8},handlelength=4)
plt.savefig('1000xht-flux'+'.pdf',bbox_inches = "tight")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [112]:
sty=style_gen()
plt.figure(dpi=80)
plt.grid()
#for i,ix in enumerate([27,68,109]):
#for i,ix in enumerate([0,41,82]):
a=[39]
nvar=43
num= len(a)
for i in range(0,num):
    a.append(a[i]+nvar)
    a.append(a[i]+nvar*2)
for i,ix in enumerate(a,):
    titl=None
#        plt.yscale('log')
#    plt.xlim(0,2450)
#    plt.ylim(-0.3,1.4)
    line_plot(x[ix],y[ix],titl,xlabel[ix],ylabel[ix],legend[ix],sty[i])

os.chdir(origdir)
#plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
#plt.legend(bbox_to_anchor=(1.,1),loc='upper left',fontsize=6)
plt.legend(loc=0,prop={'size': 7})
#plt.savefig('UabsM114-M201'+'.pdf',bbox_inches = "tight")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Diffusion comparison: compare N**2, FV, Ut of M201, M204 and M207

In [170]:
path1='/home/user/user/frontera_scratch3_mao1'
path2='/home/user/user/niagara_scratch_fherwig'
runM207='/M207-1000xht-1000xdiff-realEOS-1152'
runM204='/M204-1000xht-100xdiff-realEOS-1152'
runM201='/M201-1000xht-0xdiff-realEOS-1152'
runM114='/M114'
runs=[path1+runM201,path1+runM204,path1+runM207]
#runs=[path1+runM201,]
ieos=[1,1,1,]
iraddiff=[0,100,1000,]
#iraddiff=[0,0]
a=[[0,0,1],[1280,1280,1],[2680,2680,1]]
b=[[1400,1400,1],[2800,2800,1]]
c=[[2800,2800,1],[4200,4200,1]]
dumps=[a,b,c]

In [171]:
x,xlabel,y,ylabel,legend=compare(runs,ieos,iraddiff,dumps)

/user/frontera_scratch3_mao1/M207-1000xht-1000xdiff-realEOS-1152/prfs
run: M201
run: M201
time: 0  hours
run: M201


/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:1067: RuntimeWarning: divide by zero encountered in true_divide
  a = -(dx2)/(dx1 * (dx1 + dx2))
/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:1068: RuntimeWarning: divide by zero encountered in true_divide
  b = (dx2 - dx1) / (dx1 * dx2)
/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:1075: RuntimeWarning: invalid value encountered in add
  out[tuple(slice1)] = a * f[tuple(slice2)] + b * f[tuple(slice3)] + c * f[tuple(slice4)]
/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:1084: RuntimeWarning: divide by zero encountered in double_scalars
  out[tuple(slice1)] = (f[tuple(slice2)] - f[tuple(slice3)]) / dx_0
/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:1069: RuntimeWarning: divide by zero encountered in true_divide
  c = dx1 / (dx2 * (dx1 + dx2))
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:279: RuntimeWarning: invalid value encou

time: 1002  hours
run: M201
time: 2005  hours
run: M204
run: M204
time: 1002  hours
run: M204
time: 2005  hours
run: M207
run: M207
time: 2005  hours
run: M207
time: 3007  hours


In [172]:
for i in range(0,46):
    print(i,xlabel[i],ylabel[i],str(legend[i]))
for i in range(0,len(xlabel1)):
    x.append(x1[i])
    xlabel.append(xlabel1[i])
    y.append(y1[i])
    ylabel.append(ylabel1[i])
    legend.append(legend1[i])
print(origdir)

0 $R/\mathrm{Mm}$ $ \alpha ^2$ M201 $\alpha ^2$ 0 hrs
1 $R/\mathrm{Mm}$ $ Prad/P $ M201 $\Prad/P $ 0 hrs
2 $R/\mathrm{Mm}$ $ cp $ M201 $cp $ 0 hrs
3 $R/\mathrm{Mm}$  entropy M201 $A$ 0 hrs
4 $R/\mathrm{Mm}$  grad FV M201 $grad FV$ 0 hrs
5 $R/\mathrm{Mm}$  grad entropy  M201 $dAdR$ 0 hrs
6 $R/\mathrm{Mm}$  Uabs / Mm/s M201 $Uabs$ 0 hrs
7 $R/\mathrm{Mm}$  FV  M201 $FV$ 0 hrs
8 $R/\mathrm{Mm}$  Ut / Mm/s M201 $Ut$ 0 hrs
9 $R/\mathrm{Mm}$  Ur / Mm/s M201 $Ur$ 0 hrs
10 $R/\mathrm{Mm}$  RhoUr M201 $RhoUr$ 0 hrs
11 $R/\mathrm{Mm}$  delta M201 $delta$ 0 hrs
12 $R/\mathrm{Mm}$  P0lo M201 $P0lo$ 0 hrs
13 $R/\mathrm{Mm}$  Rho0lo M201 $Rho0lo$ 0 hrs
14 $R/\mathrm{Mm}$  P1 M201 $P1$ 0 hrs
15 $R/\mathrm{Mm}$ $\rho f$ M201 $\rho f$ 0 hrs
16 $R/\mathrm{Mm}$  $energy  / (\mathrm{L \times time}$) M201 $dE^{pot}_1/dR$ 0 hrs
17 $R/\mathrm{Mm}$  $energy  / (\mathrm{L \times time}$) M201 $dE^{int}_1/dR$ 0 hrs
18 $R/\mathrm{Mm}$  \frac{dP}{dR}/ L M201 $4\pi R^2 \rho U_R g$
19 $R/\mathrm{Mm}$  power  / L M201

In [173]:
def line_plot(x,y,title,xlabel,ylabel,lab,sty):
    if title!=None:
        plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.plot(x,y,label=lab,markersize=6,markevery=5,linestyle=sty[0], marker=sty[1], color=sty[2])
    plt.legend(loc='upper left')        
    return 0

In [175]:
sty=style_gen()
plt.figure(dpi=100)
plt.grid()
#for i,ix in enumerate([27,68,109]):
#for i,ix in enumerate([0,41,82]):
a=[32+43*i for i in range(7)]
nvar=43
num= len(a)
for i,ix in enumerate(a,):
    titl=None
#        plt.yscale('log')
    plt.xlim(1450,1580)
#    plt.ylim(-0.3,1.4)
    line_plot(x[ix],y[ix],titl,xlabel[ix],ylabel[ix],legend[ix],sty[i+3])
os.chdir(origdir)
#plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
#plt.legend(bbox_to_anchor=(1.,1),loc='upper left',fontsize=6)
plt.legend(loc="upper left",prop={'size': 7},handlelength=5)
#plt.savefig('Prad-1000xht-Krange-N2'+'.pdf',bbox_inches = "tight")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [176]:
sty=style_gen()
plt.figure(dpi=80)
plt.grid()
#for i,ix in enumerate([27,68,109]):
#for i,ix in enumerate([0,41,82]):
a=[7+43*i for i in range(7)]
nvar=43
num= len(a)
for i,ix in enumerate(a,):
    titl=None
#        plt.yscale('log')
    plt.xlim(1460,1560)
#    plt.ylim(-0.3,1.4)
    line_plot(x[ix],y[ix],titl,xlabel[ix],ylabel[ix],legend[ix],sty[i+2])
os.chdir(origdir)
#plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
#plt.legend(bbox_to_anchor=(1.,1),loc='upper left',fontsize=6)
plt.legend(loc=0,prop={'size': 7},handlelength=5)
plt.savefig('Prad-1000xht-Krange-FV'+'.pdf',bbox_inches = "tight")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [177]:
def line_plot(x,y,title,xlabel,ylabel,lab,sty):
    if title!=None:
        plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.plot(x,y,label=lab,markersize=6,markevery=30,linestyle=sty[0], marker=sty[1], color=sty[2])
    plt.legend(loc='upper left')        
    return 0

In [178]:
sty=style_gen()
plt.figure(dpi=100)
plt.grid()
#for i,ix in enumerate([27,68,109]):
#for i,ix in enumerate([0,41,82]):
a=[8+43*i for i in range(7)]
nvar=43
num= len(a)
for i,ix in enumerate(a,):
    titl=None
#        plt.yscale('log')
#    plt.xlim(1450,1600)
#    plt.ylim(-0.3,1.4)
    line_plot(x[ix],y[ix],titl,xlabel[ix],ylabel[ix],legend[ix],sty[i+2])
os.chdir(origdir)
#plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
#plt.legend(bbox_to_anchor=(1.,1),loc='upper left',fontsize=6)
plt.legend(loc=0,prop={'size': 7},handlelength=5)
#plt.savefig('Prad-1000xht-Krange-Ut'+'.pdf',bbox_inches = "tight")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Luminosity comparison: compare N**2, FV, Ut of M212, M207, M208 and M209

In [181]:
path1='/home/user/user/frontera_scratch3_mao1'
path2='/home/user/user/niagara_scratch_fherwig'
runM207='/M207-1000xht-1000xdiff-realEOS-1152'
runM208='/M208-100xht-100xdiff-realEOS-1152'
runM212='/M212-31.62xht-31.62xdiff-realEOS-1152'
runM209='/M209-3162.3xht-3162.3xdiff-realEOS-1152'
runM253='/M253-100x-896-fixKappa' 
runM114='/M114'
runs=[path1+runM212,path1+runM208,path1+runM207,path1+runM209,path1+runM253]
ieos=[1,1,1,1,1,]
iraddiff=[31.62,1000,100,3162.3,100.]
a=[[0,0,1],[700,700,1],[1540,1540,1]]
b=[[1350,1350,1],[2180,2180,1]]
c=[[1400,1400,1],[3100,3100,1]]
d=[[1350,1350,1]]
e=[[1150,1150,1]]
dumps=[a,b,c,d,e]

In [182]:
x,xlabel,y,ylabel,legend=compare(runs,ieos,iraddiff,dumps)

/user/frontera_scratch3_mao1/M253-100x-896-fixKappa/prfs
run: M212
run: M212
time: 0  hours
run: M212


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:290: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:306: RuntimeWarning: invalid value encountered in power
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:290: RuntimeWarning: divide by zero encountered in true_divide


time: 1002  hours
run: M212
time: 2205  hours
run: M208
run: M208
time: 1016  hours
run: M208
time: 2205  hours
run: M207
run: M207
time: 1002  hours
run: M207
time: 2220  hours
run: M209
run: M209
time: 1016  hours
run: M253
run: M253
time: 32943  hours


In [183]:
for i in range(0,46):
    print(i,xlabel[i],ylabel[i],str(legend[i]))
for i in range(0,len(xlabel1)):
    x.append(x1[i])
    xlabel.append(xlabel1[i])
    y.append(y1[i])
    ylabel.append(ylabel1[i])
    legend.append(legend1[i])
print(origdir)

0 $R/\mathrm{Mm}$ $ \alpha ^2$ M212 $\alpha ^2$ 0 hrs
1 $R/\mathrm{Mm}$ $ Prad/P $ M212 $\Prad/P $ 0 hrs
2 $R/\mathrm{Mm}$ $ cp $ M212 $cp $ 0 hrs
3 $R/\mathrm{Mm}$  entropy M212 $A$ 0 hrs
4 $R/\mathrm{Mm}$  grad FV M212 $grad FV$ 0 hrs
5 $R/\mathrm{Mm}$  grad entropy  M212 $dAdR$ 0 hrs
6 $R/\mathrm{Mm}$  Uabs / Mm/s M212 $Uabs$ 0 hrs
7 $R/\mathrm{Mm}$  FV  M212 $FV$ 0 hrs
8 $R/\mathrm{Mm}$  Ut / Mm/s M212 $Ut$ 0 hrs
9 $R/\mathrm{Mm}$  Ur / Mm/s M212 $Ur$ 0 hrs
10 $R/\mathrm{Mm}$  RhoUr M212 $RhoUr$ 0 hrs
11 $R/\mathrm{Mm}$  delta M212 $delta$ 0 hrs
12 $R/\mathrm{Mm}$  P0lo M212 $P0lo$ 0 hrs
13 $R/\mathrm{Mm}$  Rho0lo M212 $Rho0lo$ 0 hrs
14 $R/\mathrm{Mm}$  P1 M212 $P1$ 0 hrs
15 $R/\mathrm{Mm}$ $\rho f$ M212 $\rho f$ 0 hrs
16 $R/\mathrm{Mm}$  $energy  / (\mathrm{L \times time}$) M212 $dE^{pot}_1/dR$ 0 hrs
17 $R/\mathrm{Mm}$  $energy  / (\mathrm{L \times time}$) M212 $dE^{int}_1/dR$ 0 hrs
18 $R/\mathrm{Mm}$  \frac{dP}{dR}/ L M212 $4\pi R^2 \rho U_R g$
19 $R/\mathrm{Mm}$  power  / L M212

In [184]:
def line_plot(x,y,title,xlabel,ylabel,lab,sty):
    if title!=None:
        plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.plot(x,y,label=lab,markersize=7,markevery=5,linestyle=sty[0], marker=sty[1], color=sty[2])
    plt.legend(loc='upper left')        
    return 0

In [186]:
sty=style_gen()
plt.figure(dpi=80)
plt.grid()
a=[32+43*i for i in range(9)]
nvar=43
num= len(a)
for i,ix in enumerate(a,):
    titl=None
#        plt.yscale('log')
    plt.xlim(1450,1580)
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
#    plt.ylim(-0.3,1.4)
    line_plot(x[ix],y[ix],titl,xlabel[ix],ylabel[ix],legend[ix],sty[i+1])
os.chdir(origdir)
#plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
#plt.legend(bbox_to_anchor=(1.,1),loc='upper left',fontsize=6)
plt.legend(loc=0,prop={'size': 7},handlelength=5)
plt.savefig('Prad-K-L-N2'+'.pdf',bbox_inches = "tight")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [189]:
sty=style_gen()
plt.figure(dpi=80)
plt.grid()
a=[7+43*i for i in range(9)]
nvar=43
num= len(a)
for i,ix in enumerate(a,):
    titl=None
#        plt.yscale('log')
    plt.xlim(1460,1580)
#    plt.ylim(-0.3,1.4)
    line_plot(x[ix],y[ix],titl,xlabel[ix],ylabel[ix],legend[ix],sty[i+1])
os.chdir(origdir)
#plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
#plt.legend(bbox_to_anchor=(1.,1),loc='upper left',fontsize=6)
plt.legend(loc=0,prop={'size': 7},handlelength=5)
plt.savefig('Prad-K-L-FV'+'.pdf',bbox_inches = "tight")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [190]:
def line_plot(x,y,title,xlabel,ylabel,lab,sty):
    if title!=None:
        plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.plot(x,y,label=lab,markersize=7,markevery=30,linestyle=sty[0], marker=sty[1], color=sty[2])
    plt.legend(loc='upper left')        
    return 0

In [192]:
sty=style_gen()
plt.figure(dpi=80)
plt.grid()
a=[8+43*i for i in range(9)]
nvar=43
num= len(a)
for i,ix in enumerate(a,):
    titl=None
#        plt.yscale('log')
#    plt.xlim(1450,1600)
#    plt.ylim(-0.3,1.4)
    line_plot(x[ix],y[ix],titl,xlabel[ix],ylabel[ix],legend[ix],sty[i+1])
os.chdir(origdir)
#plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
#plt.legend(bbox_to_anchor=(1.,1),loc='upper left',fontsize=6)
plt.legend(loc=0,prop={'size': 7},handlelength=5)
plt.savefig('Prad-K-L-Ut'+'.pdf',bbox_inches = "tight")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Compare N**2, FV, Ut of M213, M207 and M210

In [143]:
path1='/home/user/user/frontera_scratch3_mao1'
path2='/home/user/user/frontera_scratch3_pwoodwar'
runM207='/M207-1000xht-1000xdiff-realEOS-1152'
runM213='/M213-1000xht-1000xdiff-realEOS-768'
runM210='/M210-1000xht-1000xdiff-realEOS-1728'
runM284='/M284-2688-Gauss3.5-48MmFWHM-realEOS-1000xhtdiff'
runs=[path1+runM213,path1+runM207,path1+runM210,path2+runM284]
ieos=[1,1,1,1,]
iraddiff=[1000,1000,1000,1000]
a=[[0,0,1],[4940,4940,1]]
b=[[4800,4800,1]]
c=[[300,300,1],[710,710,1]]
dumps=[a,b,b,c]

In [144]:
x,xlabel,y,ylabel,legend=compare(runs,ieos,iraddiff,dumps)

/user/dataA_rpod4/Huaqing/ppmstar/projects/H-core-M25/analysis_Prad-runs
run: M213
run: M213
time: 0  hours
run: M213


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:290: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:306: RuntimeWarning: invalid value encountered in power
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:290: RuntimeWarning: divide by zero encountered in true_divide


time: 3537  hours
run: M207
run: M207
time: 3437  hours
run: M210
run: M210
time: 3437  hours
run: M284


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:54: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:1067: RuntimeWarning: divide by zero encountered in true_divide
  a = -(dx2)/(dx1 * (dx1 + dx2))
/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:1068: RuntimeWarning: divide by zero encountered in true_divide
  b = (dx2 - dx1) / (dx1 * dx2)
/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:1069: RuntimeWarning: divide by zero encountered in true_divide
  c = dx1 / (dx2 * (dx1 + dx2))
/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:1075: RuntimeWarning: invalid value encountered in add
  out[tuple(slice1)] = a * f[tuple(slice2)] + b * f[tuple(slice3)] + c * f[tuple(slice4)]


run: M284
time: 1432  hours
run: M284
time: 3389  hours


In [145]:
for i in range(0,46):
    print(i,xlabel[i],ylabel[i],str(legend[i]))
for i in range(0,len(xlabel1)):
    x.append(x1[i])
    xlabel.append(xlabel1[i])
    y.append(y1[i])
    ylabel.append(ylabel1[i])
    legend.append(legend1[i])
print(origdir)

0 $R/\mathrm{Mm}$ $ \alpha ^2$ M213 $\alpha ^2$ 0 hrs
1 $R/\mathrm{Mm}$ $ Prad/P $ M213 $\Prad/P $ 0 hrs
2 $R/\mathrm{Mm}$ $ cp $ M213 $cp $ 0 hrs
3 $R/\mathrm{Mm}$  entropy M213 $A$ 0 hrs
4 $R/\mathrm{Mm}$  grad FV M213 $grad FV$ 0 hrs
5 $R/\mathrm{Mm}$  grad entropy  M213 $dAdR$ 0 hrs
6 $R/\mathrm{Mm}$  Uabs / Mm/s M213 $Uabs$ 0 hrs
7 $R/\mathrm{Mm}$  FV  M213 $FV$ 0 hrs
8 $R/\mathrm{Mm}$  Ut / Mm/s M213 $Ut$ 0 hrs
9 $R/\mathrm{Mm}$  Ur / Mm/s M213 $Ur$ 0 hrs
10 $R/\mathrm{Mm}$  RhoUr M213 $RhoUr$ 0 hrs
11 $R/\mathrm{Mm}$  delta M213 $delta$ 0 hrs
12 $R/\mathrm{Mm}$  P0lo M213 $P0lo$ 0 hrs
13 $R/\mathrm{Mm}$  Rho0lo M213 $Rho0lo$ 0 hrs
14 $R/\mathrm{Mm}$  P1 M213 $P1$ 0 hrs
15 $R/\mathrm{Mm}$ $\rho f$ M213 $\rho f$ 0 hrs
16 $R/\mathrm{Mm}$  $energy  / (\mathrm{L \times time}$) M213 $dE^{pot}_1/dR$ 0 hrs
17 $R/\mathrm{Mm}$  $energy  / (\mathrm{L \times time}$) M213 $dE^{int}_1/dR$ 0 hrs
18 $R/\mathrm{Mm}$  \frac{dP}{dR}/ L M213 $4\pi R^2 \rho U_R g$
19 $R/\mathrm{Mm}$  power  / L M213

In [146]:
def line_plot(x,y,title,xlabel,ylabel,lab,sty):
    if title!=None:
        plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.plot(x,y,label=lab,markersize=6,markevery=5,linestyle=sty[0], marker=sty[1], color=sty[2])
    plt.legend(loc='upper left')        
    return 0

In [148]:
sty=style_gen()
plt.figure(dpi=80)
plt.grid()
a=[32+43*i for i in range(6)]
nvar=43
num= len(a)
for i,ix in enumerate(a,):
    titl=None
#        plt.yscale('log')
    plt.xlim(1450,1580)
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
#    plt.ylim(-0.3,1.4)
    line_plot(x[ix],y[ix],titl,xlabel[ix],ylabel[ix],legend[ix],sty[i])
os.chdir(origdir)
#plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
#plt.legend(bbox_to_anchor=(1.,1),loc='upper left',fontsize=6)
plt.legend(loc=0,prop={'size': 7},handlelength=5)
plt.savefig('Prad-1000x-K-L-res-N2'+'.pdf',bbox_inches = "tight")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [149]:
sty=style_gen()
plt.figure(dpi=80)
plt.grid()
a=[7+43*i for i in range(6)]
nvar=43
num= len(a)
for i,ix in enumerate(a,):
    titl=None
#        plt.yscale('log')
    plt.xlim(1460,1560)
#    plt.ylim(-0.3,1.4)
    line_plot(x[ix],y[ix],titl,xlabel[ix],ylabel[ix],legend[ix],sty[i])
os.chdir(origdir)
#plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
#plt.legend(bbox_to_anchor=(1.,1),loc='upper left',fontsize=6)
plt.legend(loc=0,prop={'size': 7},handlelength=5)
plt.savefig('Prad-1000x-K-L-res-FV'+'.pdf',bbox_inches = "tight")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [150]:
def line_plot(x,y,title,xlabel,ylabel,lab,sty):
    if title!=None:
        plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.plot(x,y,label=lab,markersize=6,markevery=30,linestyle=sty[0], marker=sty[1], color=sty[2])
    plt.legend(loc='upper left')        
    return 0

In [151]:
sty=style_gen()
plt.figure(dpi=80)
plt.grid()
a=[8+43*i for i in range(6)]
nvar=43
num= len(a)
for i,ix in enumerate(a,):
    titl=None
#        plt.yscale('log')
#    plt.xlim(1450,1600)
#    plt.ylim(-0.3,1.4)
    line_plot(x[ix],y[ix],titl,xlabel[ix],ylabel[ix],legend[ix],sty[i])
os.chdir(origdir)
#plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
#plt.legend(bbox_to_anchor=(1.,1),loc='upper left',fontsize=6)
plt.legend(loc=0,prop={'size': 7},handlelength=5)
plt.savefig('Prad-1000x-K-L-res-Ut'+'.pdf',bbox_inches = "tight")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## MLT

### M201,M204,M207

In [52]:
path1='/home/user/user/frontera_scratch3_mao1'
path2='/home/user/user/niagara_scratch_fherwig'
runM207='/M207-1000xht-1000xdiff-realEOS-1152'
runM204='/M204-1000xht-100xdiff-realEOS-1152'
runM201='/M201-1000xht-0xdiff-realEOS-1152'
runs=[path1+runM201,path1+runM204,path1+runM207]
ieos=[1,1,1,]
iraddiff=[0,100,1000]
a=[[3000,3000,1],]
dumps=[a,a,a]

In [53]:
x,xlabel,y,ylabel,legend=compare(runs,ieos,iraddiff,dumps)

/user/dataA_rpod4/Huaqing/ppmstar/projects/H-core-M25/analysis_Prad-runs
run: M201
run: M201
time: 2234  hours
run: M204


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:282: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:284: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:290: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:306: RuntimeWarning: invalid value encountered in power


run: M204
time: 2148  hours
run: M207
run: M207
time: 2148  hours


In [54]:
for i in range(0,46):
    print(i,xlabel[i],ylabel[i],str(legend[i]))
for i in range(0,len(xlabel1)):
    x.append(x1[i])
    xlabel.append(xlabel1[i])
    y.append(y1[i])
    ylabel.append(ylabel1[i])
    legend.append(legend1[i])
print(origdir)

0 $R/\mathrm{Mm}$ $ \alpha ^2$ M201 $\alpha ^2$ 2234 hrs
1 $R/\mathrm{Mm}$ $ Prad/P $ M201 $\Prad/P $ 2234 hrs
2 $R/\mathrm{Mm}$ $ cp $ M201 $cp $ 2234 hrs
3 $R/\mathrm{Mm}$  entropy M201 $A$ 2234 hrs
4 $R/\mathrm{Mm}$  grad FV M201 $grad FV$ 2234 hrs
5 $R/\mathrm{Mm}$  grad entropy  M201 $dAdR$ 2234 hrs
6 $R/\mathrm{Mm}$  Uabs / Mm/s M201 $Uabs$ 2234 hrs
7 $R/\mathrm{Mm}$  FV  M201 $FV$ 2234 hrs
8 $R/\mathrm{Mm}$  Ut / Mm/s M201 $Ut$ 2234 hrs
9 $R/\mathrm{Mm}$  Ur / Mm/s M201 $Ur$ 2234 hrs
10 $R/\mathrm{Mm}$  RhoUr M201 $RhoUr$ 2234 hrs
11 $R/\mathrm{Mm}$  delta M201 $delta$ 2234 hrs
12 $R/\mathrm{Mm}$  P0lo M201 $P0lo$ 2234 hrs
13 $R/\mathrm{Mm}$  Rho0lo M201 $Rho0lo$ 2234 hrs
14 $R/\mathrm{Mm}$  P1 M201 $P1$ 2234 hrs
15 $R/\mathrm{Mm}$ $\rho f$ M201 $\rho f$ 2234 hrs
16 $R/\mathrm{Mm}$  $energy  / (\mathrm{L \times time}$) M201 $dE^{pot}_1/dR$ 2234 hrs
17 $R/\mathrm{Mm}$  $energy  / (\mathrm{L \times time}$) M201 $dE^{int}_1/dR$ 2234 hrs
18 $R/\mathrm{Mm}$  \frac{dP}{dR}/ L M201 $4\

In [57]:
sty=style_gen()
plt.figure(dpi=80)
plt.grid()
a=[28]
nvar=43
num= len(a)
for i in range(0,num):
    a.append(a[i]+nvar)
    a.append(a[i]+nvar*2)
num= len(a)
for i,ix in enumerate(a,):
    titl=None
#        plt.yscale('log')
    plt.xlim(0,1580)
    plt.ylim(-.004,.008)
    line_plot(x[ix],y[ix],titl,xlabel[ix],ylabel[ix],legend[ix],sty[i])
os.chdir(origdir)
#plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
#plt.legend(bbox_to_anchor=(1.,1),loc='upper left',fontsize=6)
plt.legend(bbox_to_anchor =(0.35, 0.95),prop={'size': 7},handlelength=5)
os.chdir('/user/dataA_rpod4/Huaqing/ppmstar/projects/H-core-M25/analysis_Prad-runs')
plt.savefig('supad'+'.pdf',bbox_inches = "tight")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [58]:
sty=style_gen()
plt.figure(dpi=80)
plt.grid()
a=[0]
nvar=43
num= len(a)
for i in range(0,num):
    a.append(a[i]+nvar)
    a.append(a[i]+nvar*2)
num= len(a)
for i,ix in enumerate(a,):
    titl=None
#        plt.yscale('log')
#    plt.xlim(0,1580)
#    plt.ylim(-.004,.008)
    line_plot(x[ix],y[ix],titl,xlabel[ix],ylabel[ix],legend[ix],sty[i])
os.chdir(origdir)
#plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
#plt.legend(bbox_to_anchor=(1.,1),loc='upper left',fontsize=6)
plt.legend(loc=0,prop={'size': 7})
plt.savefig('alphasq'+'.pdf',bbox_inches = "tight")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### M212,M208,M212

In [206]:
path1='/home/user/user/frontera_scratch3_mao1'
path2='/home/user/user/niagara_scratch_fherwig'
runM212='/M212-31.62xht-31.62xdiff-realEOS-1152'
runM208='/M208-100xht-100xdiff-realEOS-1152'
runM207='/M207-1000xht-1000xdiff-realEOS-1152'
runM209='/M209-3162.3xht-3162.3xdiff-realEOS-1152'
runs=[path1+runM212,path1+runM208,path1+runM207,path1+runM209,]
ieos=[1,1,1,1,]
iraddiff=[31.62,100,1000,3162,]
a=[[1590,1590,1]]
b=[[2000,2000,1]]
c=[[1660,1660,1],]
d= [[1000,1000,1],]
dumps=[d,a,b,c,]

In [207]:
x,xlabel,y,ylabel,legend=compare(runs,ieos,iraddiff,dumps)

/user/dataA_rpod4/Huaqing/ppmstar/projects/H-core-M25/analysis_Prad-runs
run: M212
run: M212
time: 1432  hours
run: M208


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:290: RuntimeWarning: divide by zero encountered in true_divide


run: M208
time: 1360  hours
run: M207
run: M207
time: 1432  hours
run: M209
run: M209
time: 1460  hours


In [209]:
for i in range(0,46):
    print(i,xlabel[i],ylabel[i],str(legend[i]))
for i in range(0,len(xlabel1)):
    x.append(x1[i])
    xlabel.append(xlabel1[i])
    y.append(y1[i])
    ylabel.append(ylabel1[i])
    legend.append(legend1[i])
print(origdir)

0 $R/\mathrm{Mm}$ $ \alpha ^2$ M212 $\alpha ^2$ 1432 hrs
1 $R/\mathrm{Mm}$ $ Prad/P $ M212 $\Prad/P $ 1432 hrs
2 $R/\mathrm{Mm}$ $ cp $ M212 $cp $ 1432 hrs
3 $R/\mathrm{Mm}$  entropy M212 $A$ 1432 hrs
4 $R/\mathrm{Mm}$  grad FV M212 $grad FV$ 1432 hrs
5 $R/\mathrm{Mm}$  grad entropy  M212 $dAdR$ 1432 hrs
6 $R/\mathrm{Mm}$  Uabs / Mm/s M212 $Uabs$ 1432 hrs
7 $R/\mathrm{Mm}$  FV  M212 $FV$ 1432 hrs
8 $R/\mathrm{Mm}$  Ut / Mm/s M212 $Ut$ 1432 hrs
9 $R/\mathrm{Mm}$  Ur / Mm/s M212 $Ur$ 1432 hrs
10 $R/\mathrm{Mm}$  RhoUr M212 $RhoUr$ 1432 hrs
11 $R/\mathrm{Mm}$  delta M212 $delta$ 1432 hrs
12 $R/\mathrm{Mm}$  P0lo M212 $P0lo$ 1432 hrs
13 $R/\mathrm{Mm}$  Rho0lo M212 $Rho0lo$ 1432 hrs
14 $R/\mathrm{Mm}$  P1 M212 $P1$ 1432 hrs
15 $R/\mathrm{Mm}$ $\rho f$ M212 $\rho f$ 1432 hrs
16 $R/\mathrm{Mm}$  $energy  / (\mathrm{L \times time}$) M212 $dE^{pot}_1/dR$ 1432 hrs
17 $R/\mathrm{Mm}$  $energy  / (\mathrm{L \times time}$) M212 $dE^{int}_1/dR$ 1432 hrs
18 $R/\mathrm{Mm}$  \frac{dP}{dR}/ L M212 $4\

In [210]:
sty=style_gen()
plt.figure(dpi=80)
plt.grid()
a=[28]
nvar=43
num= len(a)
for i in range(0,num):
    a.append(a[i]+nvar)
    a.append(a[i]+nvar*2)
    a.append(a[i]+nvar*3)
num= len(a)
for i,ix in enumerate(a,):
    titl=None
#        plt.yscale('log')
    plt.xlim(0,1580)
    plt.ylim(-.001,.002)
    line_plot(x[ix],y[ix],titl,xlabel[ix],ylabel[ix],legend[ix],sty[i+2])
os.chdir(origdir)
#plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
#plt.legend(bbox_to_anchor=(1.,1),loc='upper left',fontsize=6)
plt.legend(bbox_to_anchor =(0.35, 0.95),prop={'size': 7})
os.chdir('/user/dataA_rpod4/Huaqing/ppmstar/projects/H-core-M25/analysis_Prad-runs')
#plt.savefig('supad'+'.pdf',bbox_inches = "tight")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [211]:
thisdict = {
  "brand": "Ford",
  "electric": False,
  "year": 1964,
  "colors": ["red", "white", "blue"]
}
output = {**thisdict}
print(output)

{'brand': 'Ford', 'electric': False, 'year': 1964, 'colors': ['red', 'white', 'blue']}


In [212]:
a=np.zeros(5)
print(a[0:3])

[0. 0. 0.]


### M253-100x-896-fixKappa, M207

In [213]:
path1='/home/user/user/frontera_scratch3_mao1'
path2='/home/user/user/niagara_scratch_fherwig'
runM207='/M207-1000xht-1000xdiff-realEOS-1152'
runM253='/M253-100x-896-fixKappa'
runM252fix='/M252-10000x-896-fixKappa'
runM251='/M251-1000x-896'
runs=[path1+runM251,path1+runM253]
ieos=[1,1,]
iraddiff=[1000,100,]
a=[[0,0,1],[650,650,1],[850,850,1],[1050,1050,1]]
b=[[0,0,1],[650,650,1],[850,850,1],[1050,1050,1]]
dumps=[a,b]

In [419]:
x,xlabel,y,ylabel,legend=compare(runs,ieos,iraddiff,dumps)

/user/dataA_rpod4/Huaqing/ppmstar/projects/H-core-M25/analysis_Prad-runs
run: M251
run: M251
time: 0  hours
run: M251


/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:1067: RuntimeWarning: divide by zero encountered in true_divide
  a = -(dx2)/(dx1 * (dx1 + dx2))
/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:1067: RuntimeWarning: invalid value encountered in true_divide
  a = -(dx2)/(dx1 * (dx1 + dx2))
/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:1068: RuntimeWarning: divide by zero encountered in true_divide
  b = (dx2 - dx1) / (dx1 * dx2)
/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:1068: RuntimeWarning: invalid value encountered in true_divide
  b = (dx2 - dx1) / (dx1 * dx2)
/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:1069: RuntimeWarning: divide by zero encountered in true_divide
  c = dx1 / (dx2 * (dx1 + dx2))
/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:1069: RuntimeWarning: invalid value encountered in true_divide
  c = dx1 / (dx2 * (dx1 + dx2))
/usr/local/lib/python3.6/dist

time: 1862  hours
run: M251


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:288: RuntimeWarning: divide by zero encountered in true_divide


time: 2435  hours
run: M251
time: 3008  hours
run: M253
run: M253
time: 0  hours
run: M253
time: 18620  hours
run: M253
time: 24349  hours
run: M253
time: 30079  hours


In [422]:
for i in range(0,43):
    print(i,xlabel[i],ylabel[i],str(legend[i]))
for i in range(0,len(xlabel1)):
    x.append(x1[i])
    xlabel.append(xlabel1[i])
    y.append(y1[i])
    ylabel.append(ylabel1[i])
    legend.append(legend1[i])
print(origdir)

0 $R/\mathrm{Mm}$ $ \alpha ^2$ M251 $\alpha ^2$ 0 hrs
1 $R/\mathrm{Mm}$ $ Prad/P $ M251 $\Prad/P $ 0 hrs
2 $R/\mathrm{Mm}$ $ cp $ M251 $cp $ 0 hrs
3 $R/\mathrm{Mm}$  entropy M251 $A$ 0 hrs
4 $R/\mathrm{Mm}$  grad FV M251 $grad FV$ 0 hrs
5 $R/\mathrm{Mm}$  grad entropy  M251 $dAdR$ 0 hrs
6 $R/\mathrm{Mm}$  Uabs / Mm/s M251 $Uabs$ 0 hrs
7 $R/\mathrm{Mm}$  FV  M251 $FV$ 0 hrs
8 $R/\mathrm{Mm}$  Ut / Mm/s M251 $Ut$ 0 hrs
9 $R/\mathrm{Mm}$  Ur / Mm/s M251 $Ur$ 0 hrs
10 $R/\mathrm{Mm}$  RhoUr M251 $RhoUr$ 0 hrs
11 $R/\mathrm{Mm}$  delta M251 $delta$ 0 hrs
12 $R/\mathrm{Mm}$  P0lo M251 $P0lo$ 0 hrs
13 $R/\mathrm{Mm}$  Rho0lo M251 $Rho0lo$ 0 hrs
14 $R/\mathrm{Mm}$  P1 M251 $P1$ 0 hrs
15 $R/\mathrm{Mm}$ $\rho f$ M251 $\rho f$ 0 hrs
16 $R/\mathrm{Mm}$  $energy  / (\mathrm{L \times time}$) M251 $dE^{pot}_1/dR$ 0 hrs
17 $R/\mathrm{Mm}$  $energy  / (\mathrm{L \times time}$) M251 $dE^{int}_1/dR$ 0 hrs
18 $R/\mathrm{Mm}$  \frac{dP}{dR}/ L M251 $4\pi R^2 \rho U_R g$
19 $R/\mathrm{Mm}$  power  / L M251

In [423]:
def line_plot(x,y,title,xlabel,ylabel,lab,sty):
    if title!=None:
        plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.plot(x,y,label=lab,markersize=6,markevery=5,linestyle=sty[0], marker=sty[1], color=sty[2])
    plt.legend(loc='upper left')        
    return 0

In [424]:
sty=style_gen()
plt.figure(dpi=80)
plt.grid()
a=[7]
nvar=43
num= len(a)
for i in range(0,num):
    for j in range(1,8):
        a.append(a[i]+nvar*j)
num= len(a)
for i,ix in enumerate(a,):
    titl=None
#        plt.yscale('log')
#    plt.xlim(0,2500)
#    plt.ylim(-.001,.002)
    line_plot(x[ix],y[ix],titl,xlabel[ix],ylabel[ix],legend[ix],sty[i+2])
os.chdir(origdir)
#plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
#plt.legend(bbox_to_anchor=(1.,1),loc='upper left',fontsize=6)
plt.legend(bbox_to_anchor =(0.35, 0.95),prop={'size': 7},handlelength=5)
os.chdir('/user/dataA_rpod4/Huaqing/ppmstar/projects/H-core-M25/analysis_Prad-runs')
#plt.savefig('M253-FV'+'.pdf',bbox_inches = "tight")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …